In [1]:
import pandas as pd
import numpy as np
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from sqlalchemy import create_engine
from collections import Counter

In [2]:
#connie = "host= 'localhost' port='5400'"
#con = psycopg2.connect(connie)
#con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
#cur = con.cursor()
#cur.execute('''CREATE DATABASE EliteDang ''')

#con.close()
#database has been created, should not be run again

In [3]:
class database_connection():
    def __init__(self, host, name, port):
        self.host = str(host)
        self.name = str(name)
        self.port = str(port)
        self.connection = "host='%s' dbname='%s'  port='%s'" % (self.host,self.name,self.port)
        
    def insert(self, query):
        con = psycopg2.connect(self.connection)   
        cur = con.cursor()
        cur.execute(query)
        con.commit()
        con.close()     
        
    def query(self, query):
        con = psycopg2.connect(self.connection)   
        cur = con.cursor()
        cur.execute(query)
        rows = cur.fetchall()
        cols = [desc[0] for desc in cur.description]
        df = pd.DataFrame(data =rows, columns = cols)
        con.close()
        return df
    
    def upload(self, df, dfname):
        engine = create_engine('postgresql://@' + self.host + ":" + self.port + "/" + self.name)
        df.to_sql(dfname, engine)

In [4]:
elite = database_connection(host='localhost', name='elitedang', port=5400)

In [5]:
systems = pd.read_json('./systems_populated.json')
stations = pd.read_json('./stations.json')
listings = pd.read_csv('./listings.csv')
commodities = pd.read_json('./commodities.json')
factions = pd.read_csv('./factions.csv')

In [6]:
systems.head()

,allegiance,allegiance_id,controlling_minor_faction,controlling_minor_faction_id,edsm_id,government,government_id,id,is_populated,minor_faction_presences,...,reserve_type_id,security,security_id,simbad_ref,state,state_id,updated_at,x,y,z
0,Empire,2.0,1 G. Caeli Empire League,31816.0,12695.0,Patronage,144.0,1,True,"[{'minor_faction_id': 31816, 'state_id': 16, '...",...,3.0,Medium,32.0,,Boom,16.0,2018-02-21 21:38:14,80.90625,-83.53125,-30.81250
1,Federation,3.0,1 Geminorum Liberals,56677.0,3979.0,Democracy,96.0,2,True,"[{'minor_faction_id': 56677, 'state_id': 80, '...",...,1.0,Medium,32.0,1 Geminorum,None,80.0,2018-02-19 18:45:03,19.78125,3.56250,-153.81250
2,Federation,3.0,Values Party of 1 Hydrae,26307.0,15225.0,Democracy,96.0,3,True,"[{'minor_faction_id': 26307, 'state_id': 16, '...",...,1.0,High,48.0,1 Hydrae,Boom,16.0,2018-02-21 17:43:34,60.90625,28.53125,-54.90625
3,Federation,3.0,1 i Centauri Values Party,7694.0,19210.0,Democracy,96.0,4,True,"[{'minor_faction_id': 7694, 'state_id': 72, 'i...",...,2.0,Medium,32.0,,Outbreak,72.0,2018-02-21 22:48:22,38.46875,29.78125,40.46875
4,Independent,4.0,1 Kappa Cygni Movement,68556.0,10902.0,Dictatorship,112.0,5,True,"[{'minor_faction_id': 68552, 'state_id': 80, '...",...,1.0,Medium,32.0,,Civil War,64.0,2018-02-22 00:04:41,-117.75000,37.78125,11.18750


In [7]:
systems.loc[0]

allegiance                                                                 Empire
allegiance_id                                                                   2
controlling_minor_faction                                1 G. Caeli Empire League
controlling_minor_faction_id                                                31816
edsm_id                                                                     12695
government                                                              Patronage
government_id                                                                 144
id                                                                              1
is_populated                                                                 True
minor_faction_presences         [{'minor_faction_id': 31816, 'state_id': 16, '...
name                                                                   1 G. Caeli
needs_permit                                                                False
population      

In [8]:
systems.shape

(20482, 29)

In [9]:
systems.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20482 entries, 0 to 20481
Data columns (total 29 columns):
allegiance                      20472 non-null object
allegiance_id                   20472 non-null float64
controlling_minor_faction       20472 non-null object
controlling_minor_faction_id    20472 non-null float64
edsm_id                         20479 non-null float64
government                      20472 non-null object
government_id                   20472 non-null float64
id                              20482 non-null int64
is_populated                    20482 non-null bool
minor_faction_presences         20482 non-null object
name                            20482 non-null object
needs_permit                    20482 non-null bool
population                      20475 non-null float64
power                           8728 non-null object
power_state                     9140 non-null object
power_state_id                  9140 non-null float64
primary_economy              

In [10]:
stations.head()

,allegiance,allegiance_id,body_id,controlling_minor_faction_id,distance_to_star,economies,export_commodities,government,government_id,has_blackmarket,...,settlement_security_id,settlement_size,settlement_size_id,shipyard_updated_at,state,state_id,system_id,type,type_id,updated_at
0,Federation,3.0,7086578.0,13925.0,171.0,[Agriculture],"[Mineral Oil, Fruit and Vegetables, Grain]",Corporate,64.0,True,...,NaN,None,NaN,2018-02-25 18:17:16,Civil War,64.0,396,Orbis Starport,8.0,2018-02-24 20:39:41
1,Federation,3.0,3433854.0,39484.0,1080.0,"[High Tech, Refinery]","[Hydrogen Fuel, Aluminium, Advanced Catalysers]",Democracy,96.0,False,...,NaN,None,NaN,2018-02-26 02:54:11,None,80.0,773,Coriolis Starport,3.0,2018-02-26 02:54:05
2,Independent,4.0,NaN,10957.0,1834.0,[Refinery],"[Aluminium, Copper, Titanium]",Dictatorship,112.0,False,...,NaN,None,NaN,NaT,Boom,16.0,981,Civilian Outpost,1.0,2018-02-25 14:55:16
3,Federation,3.0,NaN,33245.0,228.0,[Refinery],"[Cobalt, Gold, Silver]",Corporate,64.0,False,...,NaN,None,NaN,2018-02-26 02:39:09,Boom,16.0,2236,Orbis Starport,8.0,2018-02-21 03:05:30
4,Federation,3.0,NaN,33245.0,306.0,[Refinery],"[Polymers, Copper, Titanium]",Corporate,64.0,False,...,NaN,None,NaN,2018-02-22 19:35:58,Boom,16.0,2236,Orbis Starport,8.0,2018-02-21 03:05:30


In [11]:
stations.loc[0]

allegiance                                                             Federation
allegiance_id                                                                   3
body_id                                                               7.08658e+06
controlling_minor_faction_id                                                13925
distance_to_star                                                              171
economies                                                           [Agriculture]
export_commodities                     [Mineral Oil, Fruit and Vegetables, Grain]
government                                                              Corporate
government_id                                                                  64
has_blackmarket                                                              True
has_commodities                                                              True
has_docking                                                                  True
has_market      

In [12]:
stations.shape

(67704, 39)

In [13]:
stations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67704 entries, 0 to 67703
Data columns (total 39 columns):
allegiance                      66828 non-null object
allegiance_id                   66828 non-null float64
body_id                         22939 non-null float64
controlling_minor_faction_id    66828 non-null float64
distance_to_star                64931 non-null float64
economies                       67704 non-null object
export_commodities              67704 non-null object
government                      66828 non-null object
government_id                   66828 non-null float64
has_blackmarket                 67704 non-null bool
has_commodities                 67704 non-null bool
has_docking                     67704 non-null bool
has_market                      67704 non-null bool
has_outfitting                  67704 non-null bool
has_rearm                       67704 non-null bool
has_refuel                      67704 non-null bool
has_repair                      6770

In [14]:
listings.head()

,id,station_id,commodity_id,supply,supply_bracket,buy_price,sell_price,demand,demand_bracket,collected_at
0,1,1,5,0,0.0,0,744,1159,3.0,1519787918
1,2,1,6,0,0.0,0,7474,38,3.0,1519787918
2,3,1,7,0,0.0,0,936,355,3.0,1519787918
3,12,1,8,0,0.0,0,913,12054,3.0,1519787918
4,13,1,9,0,0.0,0,1431,2121,3.0,1519787918


In [15]:
listings.loc[0]

id                1.000000e+00
station_id        1.000000e+00
commodity_id      5.000000e+00
supply            0.000000e+00
supply_bracket    0.000000e+00
buy_price         0.000000e+00
sell_price        7.440000e+02
demand            1.159000e+03
demand_bracket    3.000000e+00
collected_at      1.519788e+09
Name: 0, dtype: float64

In [16]:
listings.shape

(3187369, 10)

In [17]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3187369 entries, 0 to 3187368
Data columns (total 10 columns):
id                int64
station_id        int64
commodity_id      int64
supply            int64
supply_bracket    float64
buy_price         int64
sell_price        int64
demand            int64
demand_bracket    float64
collected_at      int64
dtypes: float64(2), int64(8)
memory usage: 243.2 MB


In [18]:
for col in listings.columns:
    print(col + ": " + str(listings[col].isnull().sum()) + " nulls")

id: 0 nulls
station_id: 0 nulls
commodity_id: 0 nulls
supply: 0 nulls
supply_bracket: 535234 nulls
buy_price: 0 nulls
sell_price: 0 nulls
demand: 0 nulls
demand_bracket: 535234 nulls
collected_at: 0 nulls


In [19]:
commodities.head()

,average_price,buy_price_lower_average,category,category_id,ed_id,id,is_non_marketable,is_rare,max_buy_price,max_sell_price,min_buy_price,min_sell_price,name,sell_price_upper_average
0,294,128,"{'id': 1, 'name': 'Chemicals'}",1,128049204.0,1,0,0,755.0,1410.0,73.0,67.0,Explosives,598
1,122,59,"{'id': 1, 'name': 'Chemicals'}",1,128049202.0,2,0,0,146.0,214.0,24.0,23.0,Hydrogen Fuel,120
2,228,63,"{'id': 1, 'name': 'Chemicals'}",1,128049203.0,3,0,0,600.0,1439.0,40.0,35.0,Mineral Oil,531
3,248,77,"{'id': 1, 'name': 'Chemicals'}",1,128049205.0,4,0,0,376.0,1583.0,42.0,38.0,Pesticides,568
4,324,141,"{'id': 2, 'name': 'Consumer Items'}",2,128049241.0,5,0,0,526.0,963.0,67.0,63.0,Clothing,600


In [20]:
commodities.loc[0]

average_price                                          294
buy_price_lower_average                                128
category                    {'id': 1, 'name': 'Chemicals'}
category_id                                              1
ed_id                                          1.28049e+08
id                                                       1
is_non_marketable                                        0
is_rare                                                  0
max_buy_price                                          755
max_sell_price                                        1410
min_buy_price                                           73
min_sell_price                                          67
name                                            Explosives
sell_price_upper_average                               598
Name: 0, dtype: object

In [21]:
commodities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 328 entries, 0 to 327
Data columns (total 14 columns):
average_price               328 non-null int64
buy_price_lower_average     328 non-null int64
category                    328 non-null object
category_id                 328 non-null int64
ed_id                       327 non-null float64
id                          328 non-null int64
is_non_marketable           328 non-null int64
is_rare                     328 non-null int64
max_buy_price               121 non-null float64
max_sell_price              174 non-null float64
min_buy_price               121 non-null float64
min_sell_price              174 non-null float64
name                        328 non-null object
sell_price_upper_average    328 non-null int64
dtypes: float64(5), int64(7), object(2)
memory usage: 48.4+ KB


In [22]:
commodities.shape

(328, 14)

In [23]:
for i in commodities.index:
    commodities.loc[i, 'category'] = commodities.loc[i, 'category']['name']
#no need to keep this in a dictionary of id: name - category id is a separate column

In [24]:
commodities.head()

,average_price,buy_price_lower_average,category,category_id,ed_id,id,is_non_marketable,is_rare,max_buy_price,max_sell_price,min_buy_price,min_sell_price,name,sell_price_upper_average
0,294,128,Chemicals,1,128049204.0,1,0,0,755.0,1410.0,73.0,67.0,Explosives,598
1,122,59,Chemicals,1,128049202.0,2,0,0,146.0,214.0,24.0,23.0,Hydrogen Fuel,120
2,228,63,Chemicals,1,128049203.0,3,0,0,600.0,1439.0,40.0,35.0,Mineral Oil,531
3,248,77,Chemicals,1,128049205.0,4,0,0,376.0,1583.0,42.0,38.0,Pesticides,568
4,324,141,Consumer Items,2,128049241.0,5,0,0,526.0,963.0,67.0,63.0,Clothing,600


In [25]:
commodities.shape

(328, 14)

In [26]:
#commodities is now free of dictionaries

In [27]:
len(listings['station_id'].unique())

47597

In [28]:
stations[stations['has_market']].shape[0]

50358

In [29]:
#listings is ultimately what we want to look at, but there are a bit under 48k station ids, compared to 50k stations
#with listed markets. Ultimately, there's not much we can do about markets that don't have listings, as I don't have
#the time necessary to go and manually try to find comprehensive listing data for a full...
stations[stations['has_market']].shape[0] - len(listings['station_id'].unique())
#number of stations through in-game methods. We'll keep the data from the stations for now, but the joined table may ignore
#a number of them purely for a lack of listing data. Limits of the dataset.

2761

In [30]:
listings[listings['supply_bracket'].isnull()]['demand_bracket'].isnull().sum()
#all supply bracket nulls have demand bracket nulls

535234

In [31]:
len(listings[listings['supply_bracket'].isnull()]['station_id'].unique())

8470

In [32]:
len(listings[listings['supply_bracket'].isnull() != True]['station_id'].unique())

39127

In [33]:
print(len(listings[listings['supply_bracket'].isnull()]['station_id'].unique()) + len(listings[listings['supply_bracket'].isnull() != True]['station_id'].unique()))
len(listings['station_id'].unique())
#no commons mean supply and demand bracket nulls are dependent on station

47597


47597

In [34]:
#can supply and demand brackets be imputed? let's check the supply category (the most obvious connections)
listings[listings['supply_bracket'] == 0]['supply'].unique()
#if supply = 0, supply_bracket is 0

array([0])

In [35]:
listings[listings['supply_bracket'] == 1]['supply'].describe()
#or perhaps not quite - supply bracket 1 also has 0s in supply, let's look closer

count    2.281700e+05
mean     1.259533e+04
std      6.343083e+04
min      0.000000e+00
25%      1.430000e+02
50%      8.750000e+02
75%      4.781000e+03
max      4.407799e+06
Name: supply, dtype: float64

In [36]:
listings[(listings['supply_bracket'] == 1) & (listings['supply'] == 0)].head()
#hmm, not just one or two. Let's see what else we can find out about the supply bracket

,id,station_id,commodity_id,supply,supply_bracket,buy_price,sell_price,demand,demand_bracket,collected_at
17295,13096,256,45,0,1.0,13443,13430,0,0.0,1510584249
50004,2297408,717,282,0,1.0,10594,10594,0,0.0,1519068794
83375,61319,1194,38,0,1.0,7970,7884,0,0.0,1519635467
83423,3920609,1194,45,0,1.0,13097,13097,0,0.0,1519635467
417726,1726954,6207,38,0,1.0,7958,7872,0,0.0,1516664324


In [37]:
listings[listings['supply_bracket'] == 0].head()

,id,station_id,commodity_id,supply,supply_bracket,buy_price,sell_price,demand,demand_bracket,collected_at
0,1,1,5,0,0.0,0,744,1159,3.0,1519787918
1,2,1,6,0,0.0,0,7474,38,3.0,1519787918
2,3,1,7,0,0.0,0,936,355,3.0,1519787918
3,12,1,8,0,0.0,0,913,12054,3.0,1519787918
4,13,1,9,0,0.0,0,1431,2121,3.0,1519787918


In [38]:
listings[listings['supply_bracket'] == 0]['buy_price'].unique()
#buy price is also all zeroes. Is that true elsewhere?

array([0])

In [39]:
listings[listings['supply_bracket'] == 1]['buy_price'].describe()

count    228170.000000
mean       2272.894706
std        3161.670982
min          10.000000
25%         235.000000
50%         748.000000
75%        2675.750000
max      124438.000000
Name: buy_price, dtype: float64

In [40]:
listings[listings['supply_bracket'] == 2]['buy_price'].describe()
#this one has a zero, worth investigating

count    338707.000000
mean       2022.069694
std        3060.476466
min           0.000000
25%         168.000000
50%         505.000000
75%        2288.000000
max       17468.000000
Name: buy_price, dtype: float64

In [41]:
listings[listings['supply_bracket'] == 3]['buy_price'].describe()

count    161554.000000
mean       1340.199203
std        2514.166103
min           1.000000
25%         109.000000
50%         375.000000
75%        1350.000000
max       16544.000000
Name: buy_price, dtype: float64

In [42]:
listings[(listings['supply_bracket'] == 2) & (listings['buy_price'] == 0)]
#this looks like an errant entry - a supply, but neither a buy nor sell price. Comfortable with turning 
#buy price = 0 into supply_bracket = 0 for imputing nulls. 

,id,station_id,commodity_id,supply,supply_bracket,buy_price,sell_price,demand,demand_bracket,collected_at
2447170,3214402,42289,4,702,2.0,0,0,0,0.0,1519516665


In [43]:
listings[listings['supply_bracket'].isnull() & (listings['buy_price'] == 0)].shape[0]

394714

In [44]:
listings[listings['supply_bracket'].isnull() & (listings['buy_price'] != 0)].shape[0]
#still to be resolved

140520

In [45]:
listings[listings['supply_bracket'] == 1]['commodity_id'].unique()

array([  2,   3,  12, 271,  13,  14,  15,  16,  18,  19,  21,  72,  73,
         5,  17,  25,  26,  27,  29,  30,  31,  66, 119, 103, 101, 287,
         7,  22,  23,  24,  37,  38,  39,  40,  41,  43,  44,  47,  48,
        49,  50,  74, 118, 282, 303, 304,   1,  42,  45,  57,  75, 278,
         4,   6,  20,  32,  36,  62,  63,  64,  65,  68,  70, 104, 306,
       300,  69,  77,  28, 305,  33,  35,  80,  78,  59,   8,   9,  54,
        55,  56,  58,  51,  52,  53,  61,  11,  81,  67,  79, 288, 307,
       116, 102, 289,  10,  34, 296, 293, 290,  60, 295, 301, 302, 294,
       291, 124, 117, 112, 111, 110, 283, 109, 122, 115, 108, 107, 105,
       121, 114])

In [46]:
listings[listings['supply_bracket'] == 2]['commodity_id'].unique() #it's not a commodity thing

array([  3,  19,  12,  72,  73,   8,   9,  11,  75,  25,  26,  27,  31,
         5,   7,   2,  33,  79,  30,  47,  52,  55,  56,  58,  45,  42,
        53,  54,  57,  39,  51,  74,  40,  48,  49,  50,  22,  23,  24,
        37,  38, 282, 303, 278,  65,  68,  70,  77,  61,  62,  63,  64,
        32, 300,   6,  80,  78,  35,  36,   4,  20,  67,  69, 305, 118,
       304,  41,  44,  17,  66,  43, 104, 306, 116,  13,  28, 271,  18,
        14,  15,  21,  16,   1, 119, 103, 101, 307, 287, 102,  29,  81,
        59,  34,  60, 289, 288,  10, 294, 296, 293, 295, 290, 301, 291,
       122, 117, 283, 115, 109, 108, 107, 105, 124, 112, 111, 110, 121])

In [47]:
#looking at buy price didn't help either - ranges were roughly the same. Supply itself?
listings[listings['supply_bracket'] == 2]['supply'].describe()

count    3.387070e+05
mean     7.854015e+04
std      3.178173e+05
min      0.000000e+00
25%      8.290000e+02
50%      5.977000e+03
75%      3.232100e+04
max      1.462078e+07
Name: supply, dtype: float64

In [48]:
listings[listings['supply_bracket'] == 3]['supply'].describe()

count    1.615540e+05
mean     3.302141e+05
std      1.065358e+06
min      0.000000e+00
25%      2.549000e+03
50%      1.894950e+04
75%      1.442188e+05
max      2.686725e+07
Name: supply, dtype: float64

In [49]:
#looking back at our previous .describe(), 3 is generally higher than 2 is generally higher than 1
#but those generally aren't definitive, and I'm not comfortable making an imputation on that
#given the missing data is by station, station information might be necessary to impute
listings[((listings['supply_bracket'].isnull()!= True) & (listings['station_id'] == 1)) & (listings['supply_bracket'] != 0)]
#picking the first station, and looking at the ones that have a supply bracket

,id,station_id,commodity_id,supply,supply_bracket,buy_price,sell_price,demand,demand_bracket,collected_at
14,24,1,75,598,3.0,15,9,0,0.0,1519787918
17,659902,1,2,661,1.0,103,98,0,0.0,1519787918


In [50]:
#what are these two commodities?
print(commodities[commodities['id'] == 2]['name'])
print(commodities[commodities['id'] == 75]['name'])

1    Hydrogen Fuel
Name: name, dtype: object
73    Biowaste
Name: name, dtype: object


In [51]:
#these are two basic commodities that pretty much every station that has a market sells. That said, 
#do they have the same supply bracket everywhere because of it? 
listings[listings['commodity_id'] == 2].head(10)
#appears not

,id,station_id,commodity_id,supply,supply_bracket,buy_price,sell_price,demand,demand_bracket,collected_at
17,659902,1,2,661,1.0,103,98,0,0.0,1519787918
34,88,3,2,97231,NaN,101,97,0,NaN,1499629225
144,155,4,2,2882454,3.0,80,76,0,0.0,1518290331
186,188,5,2,76991,1.0,104,98,0,0.0,1519782463
267,317521,6,2,342418,2.0,105,99,0,0.0,1509286102
294,302,8,2,312870,2.0,100,94,0,0.0,1519784912
394,386,9,2,1178114,2.0,97,92,0,0.0,1519276311
416,399,10,2,5881,1.0,103,98,0,0.0,1519787134
506,484,11,2,175886,2.0,92,88,0,0.0,1519704997
530,498,12,2,1078,1.0,103,98,0,0.0,1519778820


In [52]:
listings[listings['commodity_id'] == 75].head(10)

,id,station_id,commodity_id,supply,supply_bracket,buy_price,sell_price,demand,demand_bracket,collected_at
14,24,1,75,598,3.0,15,9,0,0.0,1519787918
94,150,3,75,5135,NaN,10,7,0,NaN,1499629225
171,185,4,75,0,0.0,0,111,5020559,2.0,1518290331
205,2741,5,75,0,0.0,0,325,1021943,2.0,1519782463
231,225,6,75,15569,2.0,18,13,0,0.0,1509286102
323,352,8,75,14228,2.0,99,59,0,0.0,1519784912
380,367,9,75,53277,2.0,13,10,0,0.0,1519276311
457,449,10,75,3434,3.0,18,11,0,0.0,1519787134
496,463,11,75,12688,2.0,11,8,0,0.0,1519704997
573,555,12,75,405,3.0,32,19,0,0.0,1519778820


In [53]:
stations[stations['id'] == 1] 
#they are listed in export commodities, but nothing special to guess at supply bracket

,allegiance,allegiance_id,body_id,controlling_minor_faction_id,distance_to_star,economies,export_commodities,government,government_id,has_blackmarket,...,settlement_security_id,settlement_size,settlement_size_id,shipyard_updated_at,state,state_id,system_id,type,type_id,updated_at
60000,Federation,3.0,NaN,13066.0,15460.0,[Tourism],"[Hydrogen Fuel, Biowaste]",Corporate,64.0,False,...,NaN,None,NaN,2018-02-23 18:12:23,None,80.0,18370,Ocellus Starport,7.0,2018-02-24 19:07:31


In [54]:
stations.loc[60000]

allegiance                                                             Federation
allegiance_id                                                                   3
body_id                                                                       NaN
controlling_minor_faction_id                                                13066
distance_to_star                                                            15460
economies                                                               [Tourism]
export_commodities                                      [Hydrogen Fuel, Biowaste]
government                                                              Corporate
government_id                                                                  64
has_blackmarket                                                             False
has_commodities                                                              True
has_docking                                                                  True
has_market      

In [55]:
#economies might prove important, so going to try and pull the 'tourism' economy stations to see how others work
tourism = []
for i in stations.index:
    if stations.loc[i, 'economies'] == ['Tourism']:
        tourism.append(stations.loc[i, 'id'])
set(tourism)

{1,
 174,
 703,
 742,
 812,
 825,
 859,
 1081,
 1082,
 1131,
 1307,
 1325,
 1328,
 1630,
 1658,
 1667,
 1794,
 1804,
 1806,
 2039,
 2378,
 2394,
 2440,
 2647,
 2779,
 3065,
 3067,
 3421,
 3506,
 3548,
 3637,
 3645,
 3667,
 3816,
 3838,
 3853,
 4065,
 4105,
 4128,
 4142,
 4160,
 4185,
 4186,
 4349,
 4423,
 4436,
 4440,
 4653,
 4656,
 4779,
 4801,
 4836,
 5005,
 5011,
 5249,
 5374,
 5484,
 5667,
 5768,
 5770,
 5793,
 6065,
 6108,
 6285,
 6312,
 6611,
 6612,
 6679,
 6727,
 6730,
 6731,
 6732,
 6733,
 6734,
 6735,
 6862,
 6876,
 7070,
 7076,
 7078,
 7079,
 7080,
 7083,
 7113,
 7687,
 7779,
 7961,
 7995,
 8424,
 8634,
 8830,
 8961,
 8963,
 9071,
 9072,
 9074,
 9399,
 9454,
 9573,
 9684,
 9804,
 9805,
 9969,
 9970,
 10165,
 10233,
 10236,
 10237,
 10297,
 10414,
 10496,
 10731,
 10845,
 10861,
 10893,
 10896,
 10900,
 10954,
 11191,
 11192,
 11335,
 11360,
 11411,
 11457,
 11460,
 11461,
 11601,
 11672,
 11673,
 11684,
 11704,
 11798,
 11803,
 11813,
 11926,
 11945,
 11947,
 12134,
 12183,
 

In [56]:
listings[listings['station_id'].isin(tourism) & (listings['commodity_id'] == 2)]

,id,station_id,commodity_id,supply,supply_bracket,buy_price,sell_price,demand,demand_bracket,collected_at
17,659902,1,2,661,1.0,103,98,0,0.0,1519787918
11635,8777,174,2,281980,2.0,101,96,0,0.0,1518611418
49014,36110,703,2,0,0.0,0,125,27720,1.0,1518900301
51931,38091,742,2,115171,2.0,93,89,0,0.0,1519767332
57034,41940,812,2,1083670,2.0,96,92,0,0.0,1519651282
58014,42603,825,2,1049229,2.0,95,90,0,0.0,1519468612
60360,44287,859,2,1075998,2.0,96,91,0,0.0,1519699907
75512,55586,1081,2,427969,2.0,102,97,0,0.0,1519126164
75546,55613,1082,2,585176,2.0,103,98,0,0.0,1519127318
79053,58224,1131,2,47340,1.0,113,108,0,0.0,1518080048


In [57]:
stations[stations['id'] == 742]
#despite the previous two both still being listed 'export commodities', they're in different brackets
#as a side note - limpets are another common, but not universal, because they're relevant more to ships
#than in transporting them from market to market

,allegiance,allegiance_id,body_id,controlling_minor_faction_id,distance_to_star,economies,export_commodities,government,government_id,has_blackmarket,...,settlement_security_id,settlement_size,settlement_size_id,shipyard_updated_at,state,state_id,system_id,type,type_id,updated_at
10161,Independent,4.0,8760.0,18979.0,234.0,[Tourism],"[Hydrogen Fuel, Biowaste, Limpet]",Cooperative,80.0,True,...,NaN,None,NaN,2018-02-26 02:41:32,Boom,16.0,4723,Orbis Starport,8.0,2018-02-24 13:23:57


In [6]:
#I can't find a way to otherwise impute them - further online analysis seems to imply that this
#is based off of player interactions, something I may not be able to impute purely from data. Will
#likely have to drop this column to avoid the nulls, sadly, as there are too many to just ignore
#and the correlation with stations means I can't drop the rows without losing valuable information elsewhere.
new_listings = listings.drop(['supply_bracket', 'demand_bracket', 'collected_at'], axis = 1)

In [59]:
new_listings.head(10)

,id,station_id,commodity_id,supply,buy_price,sell_price,demand
0,1,1,5,0,0,744,1159
1,2,1,6,0,0,7474,38
2,3,1,7,0,0,936,355
3,12,1,8,0,0,913,12054
4,13,1,9,0,0,1431,2121
5,14,1,12,0,0,262,287
6,15,1,30,0,0,936,269
7,16,1,31,0,0,711,440
8,17,1,33,0,0,2026,4136
9,18,1,36,0,0,7474,11


In [60]:
new_listings.shape

(3187369, 7)

In [61]:
for col in new_listings.columns:
    print(new_listings[col].isnull().sum())
#all nulls resolved - this will be the 'base' of our sql database

0
0
0
0
0
0
0


In [62]:
new_listings.set_index('id', inplace = True)

In [63]:
#elite.upload(new_listings, 'listings')
#does not need to be run again

In [64]:
elite.query("""SELECT * FROM listings
               LIMIT 20""")

,id,station_id,commodity_id,supply,buy_price,sell_price,demand
0,1,1,5,0,0,744,1159
1,2,1,6,0,0,7474,38
2,3,1,7,0,0,936,355
3,12,1,8,0,0,913,12054
4,13,1,9,0,0,1431,2121
5,14,1,12,0,0,262,287
6,15,1,30,0,0,936,269
7,16,1,31,0,0,711,440
8,17,1,33,0,0,2026,4136
9,18,1,36,0,0,7474,11


In [65]:
commodities.head()

,average_price,buy_price_lower_average,category,category_id,ed_id,id,is_non_marketable,is_rare,max_buy_price,max_sell_price,min_buy_price,min_sell_price,name,sell_price_upper_average
0,294,128,Chemicals,1,128049204.0,1,0,0,755.0,1410.0,73.0,67.0,Explosives,598
1,122,59,Chemicals,1,128049202.0,2,0,0,146.0,214.0,24.0,23.0,Hydrogen Fuel,120
2,228,63,Chemicals,1,128049203.0,3,0,0,600.0,1439.0,40.0,35.0,Mineral Oil,531
3,248,77,Chemicals,1,128049205.0,4,0,0,376.0,1583.0,42.0,38.0,Pesticides,568
4,324,141,Consumer Items,2,128049241.0,5,0,0,526.0,963.0,67.0,63.0,Clothing,600


In [66]:
commodities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 328 entries, 0 to 327
Data columns (total 14 columns):
average_price               328 non-null int64
buy_price_lower_average     328 non-null int64
category                    328 non-null object
category_id                 328 non-null int64
ed_id                       327 non-null float64
id                          328 non-null int64
is_non_marketable           328 non-null int64
is_rare                     328 non-null int64
max_buy_price               121 non-null float64
max_sell_price              174 non-null float64
min_buy_price               121 non-null float64
min_sell_price              174 non-null float64
name                        328 non-null object
sell_price_upper_average    328 non-null int64
dtypes: float64(5), int64(7), object(2)
memory usage: 48.4+ KB


In [67]:
#ed_id is, as far as I can tell, entirely superfluous for us, meaning it can be dropped
#without worrying about null values and what not.
commodities.drop('ed_id', axis = 1, inplace=True)

In [68]:
#max buy price
commodities.head()

,average_price,buy_price_lower_average,category,category_id,id,is_non_marketable,is_rare,max_buy_price,max_sell_price,min_buy_price,min_sell_price,name,sell_price_upper_average
0,294,128,Chemicals,1,1,0,0,755.0,1410.0,73.0,67.0,Explosives,598
1,122,59,Chemicals,1,2,0,0,146.0,214.0,24.0,23.0,Hydrogen Fuel,120
2,228,63,Chemicals,1,3,0,0,600.0,1439.0,40.0,35.0,Mineral Oil,531
3,248,77,Chemicals,1,4,0,0,376.0,1583.0,42.0,38.0,Pesticides,568
4,324,141,Consumer Items,2,5,0,0,526.0,963.0,67.0,63.0,Clothing,600


In [69]:
new_listings[(new_listings['commodity_id'] == 1) & (new_listings['buy_price'] > 0)]['buy_price'].max()

755

In [70]:
new_listings[(new_listings['commodity_id'] == 1) & (new_listings['buy_price'] > 0)]['buy_price'].min()

73

In [71]:
#matches our max and min prices, meaning they're calculable from the the listings themselves and are safe to be dropped
commodities.drop(['max_buy_price', 'min_buy_price', 'max_sell_price', 'min_sell_price'], axis=1, inplace=True)

In [72]:
commodities.head()

,average_price,buy_price_lower_average,category,category_id,id,is_non_marketable,is_rare,name,sell_price_upper_average
0,294,128,Chemicals,1,1,0,0,Explosives,598
1,122,59,Chemicals,1,2,0,0,Hydrogen Fuel,120
2,228,63,Chemicals,1,3,0,0,Mineral Oil,531
3,248,77,Chemicals,1,4,0,0,Pesticides,568
4,324,141,Consumer Items,2,5,0,0,Clothing,600


In [73]:
#checking the average prices to see if they match things easily calculable or worth keeping
new_listings[(new_listings['commodity_id'] == 1) & (new_listings['buy_price'] > 0)]['buy_price'].mean()

198.37373814041746

In [74]:
#doesn't appear so, we'll keep them - even if we may not actually use them
new_listings[(new_listings['commodity_id'] == 1) & (new_listings['buy_price'] > 0)]['buy_price'].median()

197.0

In [75]:
commodities.set_index('id', inplace=True)

In [76]:
#elite.upload(commodities, 'commodities')
#does not need to be run again

In [77]:
elite.query("""SELECT c.name, c.category, l.supply, l.buy_price, l.sell_price, l.demand
            FROM listings l
            INNER JOIN commodities c ON l.commodity_id = c.id
            LIMIT 10""")

,name,category,supply,buy_price,sell_price,demand
0,Clothing,Consumer Items,0,0,744,1159
1,Consumer Technology,Consumer Items,0,0,7474,38
2,Domestic Appliances,Consumer Items,0,0,936,355
3,Beer,Legal Drugs,0,0,913,12054
4,Liquor,Legal Drugs,0,0,1431,2121
5,Wine,Legal Drugs,0,0,262,287
6,Power Generators,Machinery,0,0,936,269
7,Water Purifiers,Machinery,0,0,711,440
8,Basic Medicines,Medicines,0,0,2026,4136
9,Progenitor Cells,Medicines,0,0,7474,11


In [78]:
elite.query('SELECT * FROM commodities')

,id,average_price,buy_price_lower_average,category,category_id,is_non_marketable,is_rare,name,sell_price_upper_average
0,1,294,128,Chemicals,1,0,0,Explosives,598
1,2,122,59,Chemicals,1,0,0,Hydrogen Fuel,120
2,3,228,63,Chemicals,1,0,0,Mineral Oil,531
3,4,248,77,Chemicals,1,0,0,Pesticides,568
4,5,324,141,Consumer Items,2,0,0,Clothing,600
5,6,6700,5981,Consumer Items,2,0,0,Consumer Technology,7552
6,7,530,318,Consumer Items,2,0,0,Domestic Appliances,861
7,8,215,55,Legal Drugs,3,0,0,Beer,560
8,9,632,400,Legal Drugs,3,0,0,Liquor,1124
9,10,10016,8242,Legal Drugs,3,0,0,Narcotics,10228


In [79]:
systems.head()

,allegiance,allegiance_id,controlling_minor_faction,controlling_minor_faction_id,edsm_id,government,government_id,id,is_populated,minor_faction_presences,...,reserve_type_id,security,security_id,simbad_ref,state,state_id,updated_at,x,y,z
0,Empire,2.0,1 G. Caeli Empire League,31816.0,12695.0,Patronage,144.0,1,True,"[{'minor_faction_id': 31816, 'state_id': 16, '...",...,3.0,Medium,32.0,,Boom,16.0,2018-02-21 21:38:14,80.90625,-83.53125,-30.81250
1,Federation,3.0,1 Geminorum Liberals,56677.0,3979.0,Democracy,96.0,2,True,"[{'minor_faction_id': 56677, 'state_id': 80, '...",...,1.0,Medium,32.0,1 Geminorum,None,80.0,2018-02-19 18:45:03,19.78125,3.56250,-153.81250
2,Federation,3.0,Values Party of 1 Hydrae,26307.0,15225.0,Democracy,96.0,3,True,"[{'minor_faction_id': 26307, 'state_id': 16, '...",...,1.0,High,48.0,1 Hydrae,Boom,16.0,2018-02-21 17:43:34,60.90625,28.53125,-54.90625
3,Federation,3.0,1 i Centauri Values Party,7694.0,19210.0,Democracy,96.0,4,True,"[{'minor_faction_id': 7694, 'state_id': 72, 'i...",...,2.0,Medium,32.0,,Outbreak,72.0,2018-02-21 22:48:22,38.46875,29.78125,40.46875
4,Independent,4.0,1 Kappa Cygni Movement,68556.0,10902.0,Dictatorship,112.0,5,True,"[{'minor_faction_id': 68552, 'state_id': 80, '...",...,1.0,Medium,32.0,,Civil War,64.0,2018-02-22 00:04:41,-117.75000,37.78125,11.18750


In [80]:
systems.loc[0]

allegiance                                                                 Empire
allegiance_id                                                                   2
controlling_minor_faction                                1 G. Caeli Empire League
controlling_minor_faction_id                                                31816
edsm_id                                                                     12695
government                                                              Patronage
government_id                                                                 144
id                                                                              1
is_populated                                                                 True
minor_faction_presences         [{'minor_faction_id': 31816, 'state_id': 16, '...
name                                                                   1 G. Caeli
needs_permit                                                                False
population      

In [81]:
systems.loc[0, 'minor_faction_presences']

[{'influence': 42.4182,
  'minor_faction_id': 31816,
  'state': 'Boom',
  'state_id': 16},
 {'influence': 11.9921,
  'minor_faction_id': 54517,
  'state': 'Civil War',
  'state_id': 64},
 {'influence': 0.9911000000000001,
  'minor_faction_id': 54518,
  'state': 'War',
  'state_id': 73},
 {'influence': 8.3251,
  'minor_faction_id': 54519,
  'state': 'Boom',
  'state_id': 16},
 {'influence': 11.4965,
  'minor_faction_id': 74917,
  'state': 'Civil War',
  'state_id': 64},
 {'influence': 14.4698,
  'minor_faction_id': 40897,
  'state': 'None',
  'state_id': 80},
 {'influence': 10.3072,
  'minor_faction_id': 4017,
  'state': 'War',
  'state_id': 73}]

In [82]:
factions.head()

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
0,1,39 b Draconis One,1517130437,80.0,Cooperative,4.0,Independent,80.0,None,185.0,0
1,3,39 b Draconis Silver State Corp.,1509220271,64.0,Corporate,4.0,Independent,80.0,None,185.0,0
2,4,39 b Draconis Crimson Cartel,1514419230,16.0,Anarchy,4.0,Independent,80.0,None,185.0,0
3,6,33 Bootis Worker's Party,1519229966,32.0,Communism,4.0,Independent,80.0,None,156.0,0
4,99,Friends of Addo,1519564244,80.0,Cooperative,4.0,Independent,64.0,Civil War,456.0,0


In [83]:
factions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75842 entries, 0 to 75841
Data columns (total 11 columns):
id                   75842 non-null int64
name                 75841 non-null object
updated_at           75842 non-null int64
government_id        75826 non-null float64
government           75826 non-null object
allegiance_id        75818 non-null float64
allegiance           75818 non-null object
state_id             75792 non-null float64
state                75792 non-null object
home_system_id       74780 non-null float64
is_player_faction    75842 non-null int64
dtypes: float64(4), int64(3), object(4)
memory usage: 6.4+ MB


In [84]:
factions[factions['name'].isnull()]

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
49249,69320,NaN,1519760140,96.0,Democracy,4.0,Independent,80.0,None,12749.0,1


In [85]:
systems[systems['controlling_minor_faction_id'] == 69320]

,allegiance,allegiance_id,controlling_minor_faction,controlling_minor_faction_id,edsm_id,government,government_id,id,is_populated,minor_faction_presences,...,reserve_type_id,security,security_id,simbad_ref,state,state_id,updated_at,x,y,z
1140,Independent,4.0,NULL,69320.0,10473.0,Democracy,96.0,1205,True,"[{'minor_faction_id': 9526, 'state_id': 64, 'i...",...,2.0,High,48.0,,None,80.0,2018-02-21 14:50:52,78.96875,15.62500,65.81250
2072,Independent,4.0,NULL,69320.0,6524.0,Democracy,96.0,2158,True,"[{'minor_faction_id': 4139, 'state_id': 37, 'i...",...,3.0,Low,16.0,,None,80.0,2018-02-21 21:37:40,82.28125,-1.31250,46.09375
2853,Independent,4.0,NULL,69320.0,12363.0,Democracy,96.0,2949,True,"[{'minor_faction_id': 38781, 'state_id': 16, '...",...,NaN,Low,16.0,,None,80.0,2018-02-21 08:01:48,84.53125,7.93750,61.87500
2953,Independent,4.0,NULL,69320.0,4990.0,Democracy,96.0,3049,True,"[{'minor_faction_id': 6330, 'state_id': 16, 'i...",...,2.0,Medium,32.0,CD-54 4840,None,80.0,2018-02-19 18:30:24,101.93750,15.84375,60.71875
2962,Independent,4.0,NULL,69320.0,12375.0,Democracy,96.0,3059,True,"[{'minor_faction_id': 38780, 'state_id': 16, '...",...,2.0,Low,16.0,CD-59 4616,None,80.0,2018-02-21 05:18:48,79.50000,3.87500,57.56250
4049,Independent,4.0,NULL,69320.0,12259.0,Democracy,96.0,4356,True,"[{'minor_faction_id': 11348, 'state_id': 64, '...",...,3.0,Low,16.0,DR Crucis,War,73.0,2018-02-20 18:18:14,83.25000,8.03125,53.34375
4794,Independent,4.0,NULL,69320.0,13884.0,Democracy,96.0,5106,True,"[{'minor_faction_id': 57555, 'state_id': 65, '...",...,3.0,Medium,32.0,,None,80.0,2018-02-21 11:37:56,90.62500,-2.65625,76.90625
5516,Independent,4.0,NULL,69320.0,14032.0,Democracy,96.0,5841,True,"[{'minor_faction_id': 6321, 'state_id': 64, 'i...",...,3.0,Medium,32.0,,None,80.0,2018-02-21 20:22:48,84.84375,0.43750,57.59375
7656,Independent,4.0,NULL,69320.0,13547.0,Democracy,96.0,8176,True,"[{'minor_faction_id': 11530, 'state_id': 64, '...",...,NaN,Medium,32.0,HIP 64934,None,80.0,2018-02-19 13:31:32,97.15625,10.43750,73.18750
8640,Independent,4.0,NULL,69320.0,4727.0,Democracy,96.0,9300,True,"[{'minor_faction_id': 7101, 'state_id': 80, 'i...",...,1.0,Medium,32.0,HR 5000,None,80.0,2018-02-20 03:13:10,102.18750,-5.96875,74.00000


In [86]:
#a secondary data source suggests the faction's name may actually just be "NULL" - we'll use it for now,
#and maybe see later if it actually is that manually - if not, we can adjust all this back later
#ultimately it won't matter too much, as the faction id can be used as a categorical variable, if anything
factions.loc[49249, 'name'] = 'NULL'

In [87]:
factions[factions['government'].isnull()]
#lot of common nulls

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
9544,74896,Justicars of Helios,1492028528,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9545,74904,Mercs of Mikunn,1498167262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9551,74925,Sit Nox Regnum Luna Attollat,1481696760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9560,74949,Wolf 406 Transport & Co - AEDC,1494616391,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9573,74993,Emperor`s Grace,1475489397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9576,75015,Imperial Black Wings,1476520966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9578,75025,Justicar,1475489397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9588,75068,The Canonn,1475489397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9589,75074,The Guild of Sefrys,1475489397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9591,75083,The system name Triad,1475489397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [88]:
#last 4 are special game factions - gonna check the stations they control if they have government information
#further note: they were in the game files from before the update, but only became relevant after an update
#and this particular dataset pre-dates their time in game, which is why they currently have nulls. I'm going
#to pull information from a date where they do exist and use it to fill the null values
stations[stations['controlling_minor_faction_id'] == 75875]
#may have to negotiate this with any potential time series data

,allegiance,allegiance_id,body_id,controlling_minor_faction_id,distance_to_star,economies,export_commodities,government,government_id,has_blackmarket,...,settlement_security_id,settlement_size,settlement_size_id,shipyard_updated_at,state,state_id,system_id,type,type_id,updated_at


In [89]:
new_stations = pd.read_json('./time-data/March 17/stations.json')

In [90]:
new_stations[new_stations['controlling_minor_faction_id'] == 75875]

,allegiance,allegiance_id,body_id,controlling_minor_faction_id,distance_to_star,economies,export_commodities,government,government_id,has_blackmarket,...,settlement_security_id,settlement_size,settlement_size_id,shipyard_updated_at,state,state_id,system_id,type,type_id,updated_at
67723,Federation,3.0,413974.0,75875.0,12150.0,[Prison],[],Prison,208.0,False,...,NaN,None,NaN,2018-03-10 18:02:26,None,80.0,20566,None,NaN,2018-03-10 18:02:26
67750,Federation,3.0,NaN,75875.0,4.0,[Prison],[],Prison,208.0,False,...,NaN,None,NaN,2018-03-02 22:50:32,None,80.0,98868,None,NaN,2018-03-02 22:50:33
67761,Federation,3.0,NaN,75875.0,5.0,[Prison],[],Prison,208.0,False,...,NaN,None,NaN,2018-03-15 01:48:19,None,80.0,46011,None,NaN,2018-03-15 01:41:57


In [91]:
for i in range(9996, 10000):
    factions.loc[i, 'government_id'] = 208
    factions.loc[i, 'government'] = 'Prison'

In [92]:
factions[factions['government_id'].isnull()]

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
9544,74896,Justicars of Helios,1492028528,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9545,74904,Mercs of Mikunn,1498167262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9551,74925,Sit Nox Regnum Luna Attollat,1481696760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9560,74949,Wolf 406 Transport & Co - AEDC,1494616391,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9573,74993,Emperor`s Grace,1475489397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9576,75015,Imperial Black Wings,1476520966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9578,75025,Justicar,1475489397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9588,75068,The Canonn,1475489397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9589,75074,The Guild of Sefrys,1475489397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9591,75083,The system name Triad,1475489397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [93]:
missing_gov = list(factions[factions['government_id'].isnull()].index)

In [94]:
factions[factions['name'] == "Justicar's of Helios"]
#id is a mispelling of another faction

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
5454,41396,Justicar's of Helios,1518799920,128.0,Feudal,4.0,Independent,16.0,Boom,131.0,0


In [95]:
factions[factions['name'] == "The Mercs of Mikunn"]

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
70507,8289,The Mercs of Mikunn,1519484983,112.0,Dictatorship,4.0,Independent,16.0,Boom,28346.0,1


In [96]:
factions[factions['name'].str.contains('Nox')]

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
9551,74925,Sit Nox Regnum Luna Attollat,1481696760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
51995,17524,People's Nox Independents,1519404046,96.0,Democracy,3.0,Federation,65.0,Election,14739.0,0
51996,17525,Defence Party of Nox,1519780537,112.0,Dictatorship,4.0,Independent,73.0,War,14739.0,0
51997,17526,Nox Limited,1519772716,64.0,Corporate,3.0,Federation,80.0,None,14739.0,0
51998,17527,Society of Nox,1518484726,16.0,Anarchy,4.0,Independent,80.0,None,14739.0,0


In [97]:
factions[factions['name'].str.contains('Wolf 406')]

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
9560,74949,Wolf 406 Transport & Co - AEDC,1494616391,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9595,75097,Wolf 406 Transport & Co.,1477435049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
71224,20269,Wolf 406 Transport & Co,1519763246,64.0,Corporate,1.0,Alliance,65.0,Election,19160.0,1
72124,31256,Wolf 406 Flag,1519566548,112.0,Dictatorship,4.0,Independent,16.0,Boom,19160.0,0
72125,31257,New Wolf 406 Values Party,1518613233,96.0,Democracy,4.0,Independent,73.0,War,19160.0,0
72794,38628,Wolf 406 Raiders,1518959426,16.0,Anarchy,4.0,Independent,16.0,Boom,19160.0,0


In [98]:
factions[factions['name'] == "Emperor's Grace"]

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
57063,53045,Emperor's Grace,1518202760,144.0,Patronage,2.0,Empire,80.0,None,14735.0,0


In [99]:
factions[factions['name'] == "Imperial Black Wing's"]

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
9691,75281,Imperial Black Wing's,1519169171,64.0,Corporate,2.0,Empire,16.0,Boom,NaN,0


In [100]:
factions[factions['name'] == "Canonn"]

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
70860,14271,Canonn,1519477599,80.0,Cooperative,4.0,Independent,16.0,Boom,18454.0,1


In [101]:
factions[factions['name'] == "The Guild of Sefry"]

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
1015,8290,The Guild of Sefry,1519767424,16.0,Anarchy,4.0,Independent,16.0,Boom,NaN,1


In [102]:
factions[factions['name'].str.contains('Triad')]

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
9591,75083,The system name Triad,1475489397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9664,75224,HR 6565 Triad,1519702022,144.0,Patronage,2.0,Empire,16.0,Boom,NaN,0


In [103]:
factions[factions['name'].str.contains('Furries')]

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
9702,75303,Furries of Sol,1490202101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [104]:
def check_defunct(faction_id):
    if len(stations[stations['controlling_minor_faction_id'] == faction_id].index) == 0:
        if len(systems[systems['controlling_minor_faction_id'] == faction_id].index) == 0:
            return True
        else:
            return False
    else:
        return False

In [105]:
#all but Nox Regnum and Furries of Sol can be explained away as typos. Do those show up elsewhere?
check_defunct(74925)

True

In [106]:
#Nope. Feel safe dropping these
check_defunct(75303)

True

In [107]:
factions.drop(missing_gov, inplace=True)

In [108]:
factions[factions['government'].isnull()]
#nulls cleared!

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction


In [109]:
factions['government_id'] = factions['government_id'].astype(int)
factions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75830 entries, 0 to 75841
Data columns (total 11 columns):
id                   75830 non-null int64
name                 75830 non-null object
updated_at           75830 non-null int64
government_id        75830 non-null int64
government           75830 non-null object
allegiance_id        75818 non-null float64
allegiance           75818 non-null object
state_id             75792 non-null float64
state                75792 non-null object
home_system_id       74780 non-null float64
is_player_faction    75830 non-null int64
dtypes: float64(3), int64(4), object(4)
memory usage: 6.9+ MB


In [110]:
factions[factions['allegiance'].isnull()]

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
3684,27156,HIP 80364 party,1487514746,64,Corporate,NaN,NaN,80.0,None,NaN,0
9536,74868,Crimson Mining and Salvage,1492995640,64,Corporate,NaN,NaN,NaN,NaN,NaN,1
9537,74873,Dukes of Mikunn,1498167262,128,Feudal,NaN,NaN,16.0,Boom,NaN,1
9540,74876,Elite Truckers,1477232247,64,Corporate,NaN,NaN,NaN,NaN,NaN,1
9541,74885,Galactic Frontier Defense Initiative,1492084374,112,Dictatorship,NaN,NaN,NaN,NaN,NaN,1
9546,74908,Nova Elite,1492339948,112,Dictatorship,NaN,NaN,NaN,NaN,NaN,1
9552,74927,"Strategic Wings, Order and Retaliation Division",1475489397,80,Cooperative,NaN,NaN,NaN,NaN,NaN,1
9555,74936,The Great Empire of Star,1475489397,112,Dictatorship,NaN,NaN,NaN,NaN,NaN,1
9556,74938,The Phoenix Groop,1492010487,96,Democracy,NaN,NaN,NaN,NaN,NaN,1
9558,74941,The Void's Pirate Corporation,1478878003,64,Corporate,NaN,NaN,NaN,NaN,NaN,1


In [111]:
missing_alleg = list(factions[factions['allegiance_id'].isnull()].index)

In [112]:
#HIP 80364 Party is defunct
check_defunct(27156)

True

In [113]:
#Crimson Mining and Salvage should be Crimson Mining and Salvage Corporation
factions[factions['name'] == 'Crimson Mining and Salvage Corporation']

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
31802,10761,Crimson Mining and Salvage Corporation,1518869363,64,Corporate,4.0,Independent,73.0,War,8464.0,1


In [114]:
#Dukes of Mikunn is a duplicate
factions[factions['name'] == 'The Dukes of Mikunn']

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
50016,363,The Dukes of Mikunn,1519706078,128,Feudal,4.0,Independent,65.0,Election,13649.0,1


In [115]:
#Elite Truckers is defunct
check_defunct(74876)

True

In [116]:
#Galactic Frontier Defense Initiative is defunct
check_defunct(74885)

True

In [117]:
#Nova Elite is defunct
check_defunct(74908)

True

In [118]:
#Strategic Wings, Order and Retaliation Division is defunct
check_defunct(74927)

True

In [119]:
#The Great Empire of Star is defunct
check_defunct(74936)

True

In [120]:
#The Phoenix Groop is misspelled
factions[factions['name'] == 'The Phoenix Group']

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
19295,70785,The Phoenix Group,1519060243,96,Democracy,4.0,Independent,64.0,Civil War,3555.0,1


In [121]:
#The Void's Pirate Corporation is defunct
check_defunct(74941)

True

In [122]:
#Trading Alliance is defunct
check_defunct(74942)

True

In [123]:
#Xbone PA is defunct
check_defunct(74951)

True

In [124]:
#dropping them all
factions.drop(missing_alleg, inplace=True)

In [125]:
#nulls cleared!
factions[factions['allegiance'].isnull()]

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction


In [126]:
factions['allegiance_id'] = factions['allegiance_id'].astype(int)
factions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75818 entries, 0 to 75841
Data columns (total 11 columns):
id                   75818 non-null int64
name                 75818 non-null object
updated_at           75818 non-null int64
government_id        75818 non-null int64
government           75818 non-null object
allegiance_id        75818 non-null int64
allegiance           75818 non-null object
state_id             75790 non-null float64
state                75790 non-null object
home_system_id       74780 non-null float64
is_player_faction    75818 non-null int64
dtypes: float64(2), int64(5), object(4)
memory usage: 6.9+ MB


In [127]:
factions[factions['state'].isnull()]

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
366,2614,Silver Control Ltdmpany,1495148436,64,Corporate,3,Federation,NaN,NaN,NaN,0
796,6546,Mafia of Almar,1475489062,16,Anarchy,4,Independent,NaN,NaN,NaN,0
1276,10363,Jaque,1495289082,80,Cooperative,4,Independent,NaN,NaN,NaN,0
6467,49235,Freedom Party of Naraka,1492011223,112,Dictatorship,4,Independent,NaN,NaN,NaN,0
6468,49236,Naraka Company,1492011223,64,Corporate,3,Federation,NaN,NaN,NaN,0
6469,49237,Conservatives of Naraka,1492011223,112,Dictatorship,4,Independent,NaN,NaN,NaN,0
8601,66788,Arti Guardians of Tradition,1475489354,144,Patronage,2,Empire,NaN,NaN,NaN,0
8603,66790,Democrats of Arti,1475489354,96,Democracy,4,Independent,NaN,NaN,NaN,0
8604,66791,Arti Silver Cartel,1475489354,16,Anarchy,4,Independent,NaN,NaN,NaN,0
8605,66792,New Arti Freedom Party,1475489354,112,Dictatorship,4,Independent,NaN,NaN,NaN,0


In [128]:
missing_state = list(factions[factions['state_id'].isnull()].index)

In [129]:
#Silver Control Ltdmpany should be Silver Control Ltd
factions[factions['name'] == 'Silver Control Ltd']

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
9849,75513,Silver Control Ltd,1517882745,64,Corporate,3,Federation,80.0,None,NaN,0


In [130]:
#Mafia of Almar is defunct
check_defunct(6546)

True

In [131]:
#Jaque should be Jaques
factions[factions['name'] == 'Jaques']

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
75732,75103,Jaques,1519490143,80,Cooperative,4,Independent,65.0,Election,633211.0,0


In [132]:
#Naraka factions have changed, all these are defunct
for naraka in range(49235, 49238):
    print(check_defunct(naraka))

True
True
True


In [133]:
#Arti factions have changed - 66789 is not actually within the list, but makes the iteration easier
for arti in range(66788, 66793):
    print(check_defunct(arti))
#nulls are all defunct

True
False
True
True
True


In [134]:
factions[factions['id'] == 66789]

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
8602,66789,Gold Comms Company,1483404182,64,Corporate,4,Independent,80.0,None,NaN,0


In [135]:
#303rd Privateer Group is defunct
check_defunct(74846)

True

In [136]:
#Armata Brancaleone owns a station in Maya and in a boom - filling in from outside source (inara.cz)
factions.loc[9533]

id                                74851
name                 Armata Brancaleone
updated_at                   1475489396
government_id                       128
government                       Feudal
allegiance_id                         3
allegiance                   Federation
state_id                            NaN
state                               NaN
home_system_id                      NaN
is_player_faction                     1
Name: 9533, dtype: object

In [137]:
systems[systems['name'] == 'Maya']

,allegiance,allegiance_id,controlling_minor_faction,controlling_minor_faction_id,edsm_id,government,government_id,id,is_populated,minor_faction_presences,...,reserve_type_id,security,security_id,simbad_ref,state,state_id,updated_at,x,y,z
12666,Federation,3.0,Maya Purple Universal Inc,1063.0,9028.0,Corporate,64.0,13432,True,"[{'minor_faction_id': 1062, 'state_id': 65, 'i...",...,3.0,High,48.0,,Boom,16.0,2018-02-21 23:33:11,-5.375,42.65625,-58.46875


In [138]:
factions[factions['state'] == 'Boom'].head()

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
5,100,Addo Silver Federal Holdings,1519153494,64,Corporate,3,Federation,16.0,Boom,456.0,0
6,101,Addo Dominion,1519662844,128,Feudal,2,Empire,16.0,Boom,456.0,0
7,102,Atroco Emperor's Grace,1519278181,144,Patronage,2,Empire,16.0,Boom,1455.0,0
11,134,Social BD+37 2625 Values Party,1518414330,96,Democracy,1,Alliance,16.0,Boom,1987.0,0
19,148,Anahit Gold Brothers,1519687813,16,Anarchy,4,Independent,16.0,Boom,877.0,0


In [139]:
factions.loc[9533, 'state'] = 'Boom'
factions.loc[9533, 'state_id'] = 16
factions.loc[9533, 'home_system_id'] = 13432
missing_state.remove(9533)

In [140]:
factions.loc[9533]

id                                74851
name                 Armata Brancaleone
updated_at                   1475489396
government_id                       128
government                       Feudal
allegiance_id                         3
allegiance                   Federation
state_id                             16
state                              Boom
home_system_id                    13432
is_player_faction                     1
Name: 9533, dtype: object

In [141]:
#Brani Defense Force is defunct
check_defunct(74857)

True

In [142]:
#Elite Outlaws is defunct
check_defunct(74875)

True

In [143]:
#Group of Hungarian Badasses is defunct
check_defunct(74887)

True

In [144]:
#Hitech Aerospatial Limited is defunct
check_defunct(74890)

True

In [145]:
#Nerthus Mob should be Mob of Nerthus
factions[factions['name'] == 'Mob of Nerthus']

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
59666,73163,Mob of Nerthus,1519689817,16,Anarchy,4,Independent,80.0,None,14324.0,0


In [146]:
#Revolutionary of Bugulmara Future should be Revolutionary Bugulmara Future
factions[factions['name'] == 'Revolutionary Bugulmara Future']

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
13026,25651,Revolutionary Bugulmara Future,1509482112,96,Democracy,4,Independent,80.0,None,2652.0,0


In [147]:
#Wuy Jugun Collective Cooperative should be Wuy Jugun Collective 
factions[factions['name'] == 'Wuy jugun Collective']

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
71411,23195,Wuy jugun Collective,1518955448,80,Cooperative,4,Independent,16.0,Boom,19337.0,0


In [148]:
#Confederate Pucci Pirates is defunct
check_defunct(75584)

True

In [149]:
#Pirate is probably a mistaken entry - there are a huge number of 'Pirate' factions, but they're all
#"Pirates of " + the home system of wherever they're based out of, not just generically 'Pirate'
check_defunct(75792)

True

In [150]:
for cd_44_1695 in range(38748, 38753):
    print(check_defunct(cd_44_1695))
#first one is NOT defunct, rest are

False
True
True
True
True


In [151]:
factions.loc[14911, 'state'] = 'None'
factions.loc[14911, 'state_id'] = 80
missing_state.remove(14911)

In [152]:
factions.loc[14911]

id                                                  38748
name                 Movement for CD-44 1695 Values Party
updated_at                                     1475489212
government_id                                          96
government                                      Democracy
allegiance_id                                           3
allegiance                                     Federation
state_id                                               80
state                                                None
home_system_id                                       3023
is_player_faction                                       0
Name: 14911, dtype: object

In [153]:
#United HIP 54530 Nationalists is NOT defunct, but also in a None state
factions.loc[35203, 'state'] = 'None'
factions.loc[35203, 'state_id'] = 80
missing_state.remove(35203)

In [154]:
factions.loc[35203]

id                                           34817
name                 United HIP 54530 Nationalists
updated_at                              1475489193
government_id                                  112
government                            Dictatorship
allegiance_id                                    4
allegiance                             Independent
state_id                                        80
state                                         None
home_system_id                                7923
is_player_faction                                0
Name: 35203, dtype: object

In [155]:
#HIP 54530 Gold Society should be HIP 54530 Gold Universal Group
factions[factions['name'] == 'HIP 54530 Gold Universal Group']

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
35202,34815,HIP 54530 Gold Universal Group,1475489613,64,Corporate,3,Federation,80.0,None,7923.0,0


In [156]:
factions.drop(missing_state, inplace=True)

In [157]:
factions.loc[[9533, 14911, 35203]]
#making sure the filled-in sets remained

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
9533,74851,Armata Brancaleone,1475489396,128,Feudal,3,Federation,16.0,Boom,13432.0,1
14911,38748,Movement for CD-44 1695 Values Party,1475489212,96,Democracy,3,Federation,80.0,None,3023.0,0
35203,34817,United HIP 54530 Nationalists,1475489193,112,Dictatorship,4,Independent,80.0,None,7923.0,0


In [158]:
factions[factions['state'].isnull()]
#nulls cleared!

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction


In [159]:
factions['state_id'] = factions['state_id'].astype(int)
factions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75793 entries, 0 to 75841
Data columns (total 11 columns):
id                   75793 non-null int64
name                 75793 non-null object
updated_at           75793 non-null int64
government_id        75793 non-null int64
government           75793 non-null object
allegiance_id        75793 non-null int64
allegiance           75793 non-null object
state_id             75793 non-null int64
state                75793 non-null object
home_system_id       74776 non-null float64
is_player_faction    75793 non-null int64
dtypes: float64(1), int64(6), object(4)
memory usage: 9.4+ MB


In [160]:
factions[factions['home_system_id'].isnull()]

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
51,428,Eravarenth Alliance,1519475832,48,Confederacy,3,Federation,16,Boom,NaN,0
74,579,Bragarijin Nobles,1519630648,128,Feudal,2,Empire,73,War,NaN,0
78,588,Federal Congress,1519649099,96,Democracy,3,Federation,16,Boom,NaN,0
79,589,Sol Workers' Party,1519568380,96,Democracy,3,Federation,48,Civil Unrest,NaN,0
80,591,Sol Constitution Party,1518703031,96,Democracy,3,Federation,16,Boom,NaN,0
81,593,Sol Nationalists,1519568384,96,Democracy,3,Federation,16,Boom,NaN,0
140,1079,Order of Ethgreze,1519649039,112,Dictatorship,4,Independent,16,Boom,NaN,0
176,1265,Nobles of Hecate,1519761386,128,Feudal,4,Independent,80,None,NaN,0
211,1482,Eranin Peoples Party,1519649012,32,Communism,4,Independent,37,Famine,NaN,0
218,1557,Nai No Kami Electronics Solutions,1517060169,64,Corporate,4,Independent,80,None,NaN,0


In [161]:
#this is way too many to check manually, but just from casual glance can recognize that most of these are
#actual factions, so dropping them isn't really the right option. Fortunately, most of the factions have the
#home system name in them - while it won't kill every null, it should slim things down a bit
def find_system(faction_id):
    name = factions.loc[factions[factions['id'] == faction_id].index[0], 'name']
    namelist = name.split()
    for n in namelist:
        for s in systems.index:
            if n == systems.loc[s, 'name']:
                factions.loc[factions[factions['id'] == faction_id].index[0], 'home_system_id'] = systems.loc[s, 'id']

In [162]:
find_system(593)
factions.loc[81]

id                                593
name                 Sol Nationalists
updated_at                 1519568384
government_id                      96
government                  Democracy
allegiance_id                       3
allegiance                 Federation
state_id                           16
state                            Boom
home_system_id                  17072
is_player_faction                   0
Name: 81, dtype: object

In [163]:
find_system(428)
factions.loc[51]

id                                   428
name                 Eravarenth Alliance
updated_at                    1519475832
government_id                         48
government                   Confederacy
allegiance_id                          3
allegiance                    Federation
state_id                              16
state                               Boom
home_system_id                      4614
is_player_faction                      0
Name: 51, dtype: object

In [164]:
for a in list(factions[factions['home_system_id'].isnull()]['id']):
    find_system(a)

In [165]:
factions[factions['home_system_id'].isnull()]

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
78,588,Federal Congress,1519649099,96,Democracy,3,Federation,16,Boom,NaN,0
218,1557,Nai No Kami Electronics Solutions,1517060169,64,Corporate,4,Independent,80,None,NaN,0
250,1812,HIP 93377 Blue Brotherhood,1519762667,16,Anarchy,4,Independent,80,None,NaN,0
297,2154,Crimson Allied Co,1517406164,64,Corporate,3,Federation,80,None,NaN,0
317,2310,BD+42 3917 Free,1519482973,96,Democracy,3,Federation,73,War,NaN,0
364,2597,LP 176-55 Domain,1518529365,128,Feudal,4,Independent,16,Boom,NaN,0
365,2605,Liberty Party of 78 Ursae Majoris,1519218994,112,Dictatorship,4,Independent,96,Retreat,NaN,0
368,2639,Lei Cherna Jet Vision Corp.,1519239680,64,Corporate,4,Independent,80,None,NaN,0
375,2719,Bureau of 29 Arietis League,1517936760,112,Dictatorship,4,Independent,16,Boom,NaN,0
384,2787,Autocracy of LP 245-10,1519476858,112,Dictatorship,4,Independent,64,Civil War,NaN,0


In [166]:
factions.loc[78, 'home_system_id'] = 17072
factions.loc[619, 'home_system_id'] = 16827
#I saw this on the list - this one comes from game knowledge

In [167]:
#so this is going to be a monster function - basically iterating through word combinations of names
#until it finds a combination that matches a system name. Fortunately, names will be sequential, limiting
#the permutations. Break statements are included to try and preserve computational time.
def find_multiword_systems(faction_id):
    name = factions.loc[factions[factions['id'] == faction_id].index[0], 'name']
    namelist = name.split()
    twowordlist = []
    threewordlist = []
    if len(namelist) > 1:
        for n in range(0, len(namelist) - 1):
            twowordlist.append(namelist[n] + " " + namelist[n+1])
        if len(namelist) > 2: 
            for n in range(0, len(namelist) - 2):
                threewordlist.append(namelist[n] + " " + namelist[n+1] + " " + namelist[n+2])
    if twowordlist:
        for w in twowordlist:
            for s in systems.index:
                if w == systems.loc[s, 'name']:
                    factions.loc[factions[factions['id'] == faction_id].index[0], 'home_system_id'] = systems.loc[s, 'id']
#                    break
#            if factions.loc[factions[factions['id'] == faction_id].index[0], 'home_system_id'] != np.nan:
#                break
#    if factions.loc[factions[factions['id'] == faction_id].index[0], 'home_system_id'] == np.nan:
    if threewordlist:
        for w in threewordlist:
            for s in systems.index:
                if w == systems.loc[s, 'name']:
                    factions.loc[factions[factions['id'] == faction_id].index[0], 'home_system_id'] = systems.loc[s, 'id']
#                        break
#                if factions.loc[factions[factions['id'] == faction_id].index[0], 'home_system_id'].isnull() != np.nan:
#                    break

In [168]:
find_multiword_systems(2605)

In [169]:
factions.loc[365]

id                                                2605
name                 Liberty Party of 78 Ursae Majoris
updated_at                                  1519218994
government_id                                      112
government                                Dictatorship
allegiance_id                                        4
allegiance                                 Independent
state_id                                            96
state                                          Retreat
home_system_id                                     327
is_player_faction                                    0
Name: 365, dtype: object

In [170]:
for l in list(factions[factions['home_system_id'].isnull()]['id']):
    find_multiword_systems(l)

In [171]:
factions[factions['home_system_id'].isnull()]

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
297,2154,Crimson Allied Co,1517406164,64,Corporate,3,Federation,80,None,NaN,0
638,5335,Tainan Legon,1493071088,96,Democracy,1,Alliance,67,Expansion,NaN,0
938,7510,Gold Federal Incorporated,1514539835,64,Corporate,4,Independent,65,Election,NaN,0
1015,8290,The Guild of Sefry,1519767424,16,Anarchy,4,Independent,16,Boom,NaN,1
1087,8991,Blue Allied Partners,1516186161,64,Corporate,4,Independent,80,None,NaN,0
1872,14653,Sophie Corp,1516911231,64,Corporate,4,Independent,73,War,NaN,0
2881,21829,Gold Fortune Limited,1519653811,64,Corporate,4,Independent,16,Boom,NaN,0
3685,27157,Drty,1511663267,64,Corporate,4,Independent,73,War,NaN,0
3998,29278,The Vail,1518188091,96,Democracy,4,Independent,80,None,NaN,0
5234,39486,Gold United Systems,1519132750,64,Corporate,4,Independent,73,War,NaN,0


In [172]:
factions[factions['home_system_id'].isnull() & (factions['government'] == 'Engineer')]

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
9600,75104,Felicity Farseer,1507459464,192,Engineer,3,Federation,80,None,NaN,0
9694,75287,Marco Qwent,1507495823,192,Engineer,4,Independent,80,None,NaN,0
9697,75292,The Dweller,1507327395,192,Engineer,4,Independent,80,None,NaN,0
9699,75300,Lori Jameson,1507327034,192,Engineer,4,Independent,80,None,NaN,0
9700,75301,Colonel Bris Dekker,1507483033,192,Engineer,3,Federation,80,None,NaN,0


In [173]:
#engineers are special characters with fixed locations - inputting them here
factions.loc[9600, 'home_system_id'] = 4079
factions.loc[9694, 'home_system_id'] = 16977
factions.loc[9697, 'home_system_id'] = 19341
factions.loc[9699, 'home_system_id'] = 16827
factions.loc[9700, 'home_system_id'] = 17072

In [174]:
#a few of these are clustered together, just minor spelling changes or what not that can be quickly resolved
for r in range(5455, 5459):
    factions.loc[r, 'home_system_id'] = 30392

In [175]:
for r in range(9993, 9996):
    factions.loc[r, 'home_system_id'] = 63013

In [176]:
for r in range(8768, 8772):
    factions.loc[r, 'home_system_id'] = 21114

In [496]:
factions[factions['home_system_id'].isnull()]

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
638,5335,Tainan Legon,1493071088,96,Democracy,1,Alliance,67,Expansion,NaN,0
3685,27157,Drty,1511663267,64,Corporate,4,Independent,73,War,NaN,0
9586,75053,Reserve Imperial Logistic Auxilary Fleet,1519762506,144,Patronage,2,Empire,16,Boom,NaN,1
9619,75138,Green Party,1477541411,96,Democracy,4,Independent,80,None,NaN,0
9634,75171,Milannov's Reavers,1478810379,16,Anarchy,4,Independent,16,Boom,NaN,0
9674,75248,Eol Prou Group (Colonia),1486600123,64,Corporate,4,Independent,80,None,NaN,0


In [178]:
#I couldn't figure out any tricks to impute, and yet I can't just drop the data, unfortunately
#so I'm left with manual input using another third-party source (inara.cz) - what a pain.
#Fortunately, once entered, the origin system doesn't change, meaning any transformation done here
#could also be done on any subsequent entries that might pop up in future dates from the faction.csv file
factions.loc[5859, 'home_system_id'] = 17684
factions.loc[297, 'home_system_id'] = 18904
factions.loc[938, 'home_system_id'] = 55053
factions.loc[1015, 'home_system_id'] = 28346
factions.loc[1087, 'home_system_id'] = 16353 
#
factions.loc[1872, 'home_system_id'] = 17095
factions.loc[2881, 'home_system_id'] = 3906
factions.loc[3998, 'home_system_id'] = 5282
factions.loc[5234, 'home_system_id'] = 5149
factions.loc[5532, 'home_system_id'] = 16827 
#
factions.loc[6679, 'home_system_id'] = 17829
factions.loc[7263, 'home_system_id'] = 9961
factions.loc[7558, 'home_system_id'] = 3137
factions.loc[7940, 'home_system_id'] = 17102
factions.loc[8602, 'home_system_id'] = 3616854
#
factions.loc[9585, 'home_system_id'] = 11393
factions.loc[9596, 'home_system_id'] = 19039
factions.loc[9597, 'home_system_id'] = 20656
factions.loc[9598, 'home_system_id'] = 44155
factions.loc[9602, 'home_system_id'] = 21180
#
factions.loc[9605, 'home_system_id'] = 694257
factions.loc[9618, 'home_system_id'] = 2689
factions.loc[9621, 'home_system_id'] = 3071
factions.loc[9622, 'home_system_id'] = 1487905
factions.loc[9624, 'home_system_id'] = 10917791

In [179]:
factions.loc[9629, 'home_system_id'] = 310
factions.loc[9631, 'home_system_id'] = 10918776
factions.loc[9632, 'home_system_id'] = 10941747
factions.loc[9633, 'home_system_id'] = 5042
factions.loc[9635, 'home_system_id'] = 10918334
#
factions.loc[9636, 'home_system_id'] = 10924175
factions.loc[9637, 'home_system_id'] = 18060
factions.loc[9638, 'home_system_id'] = 676342
factions.loc[9639, 'home_system_id'] = 10918695
factions.loc[9640, 'home_system_id'] = 24948
#
factions.loc[9642, 'home_system_id'] = 9690
factions.loc[9646, 'home_system_id'] = 15252
factions.loc[9647, 'home_system_id'] = 2682
factions.loc[9649, 'home_system_id'] = 2844
factions.loc[9651, 'home_system_id'] = 17252
#
factions.loc[9653, 'home_system_id'] = 972
factions.loc[9654, 'home_system_id'] = 21122
factions.loc[9655, 'home_system_id'] = 19341
factions.loc[9657, 'home_system_id'] = 10123
factions.loc[9658, 'home_system_id'] = 4600
#
factions.loc[9659, 'home_system_id'] = 19572
factions.loc[9661, 'home_system_id'] = 699
factions.loc[9666, 'home_system_id'] = 5455
factions.loc[9667, 'home_system_id'] = 12039
factions.loc[9668, 'home_system_id'] = 2564

In [180]:
factions.loc[9670, 'home_system_id'] = 4454
factions.loc[9671, 'home_system_id'] = 12133
factions.loc[9673, 'home_system_id'] = 2399370
factions.loc[9675, 'home_system_id'] = 12267
factions.loc[9677, 'home_system_id'] = 9654
#
factions.loc[9678, 'home_system_id'] = 19647
factions.loc[9680, 'home_system_id'] = 12931
factions.loc[9681, 'home_system_id'] = 13982
factions.loc[9682, 'home_system_id'] = 2241
factions.loc[9683, 'home_system_id'] = 16245
#
factions.loc[9684, 'home_system_id'] = 11899
factions.loc[9685, 'home_system_id'] = 19667
factions.loc[9686, 'home_system_id'] = 19897
factions.loc[9688, 'home_system_id'] = 8020
factions.loc[9689, 'home_system_id'] = 18966
#
factions.loc[9690, 'home_system_id'] = 18975
factions.loc[9691, 'home_system_id'] = 833
factions.loc[9692, 'home_system_id'] = 1350054
factions.loc[9693, 'home_system_id'] = 18828
factions.loc[9698, 'home_system_id'] = 8944
#
factions.loc[9705, 'home_system_id'] = 11141
factions.loc[9706, 'home_system_id'] = 10377
factions.loc[9707, 'home_system_id'] = 131175
factions.loc[9708, 'home_system_id'] = 5826281
factions.loc[9710, 'home_system_id'] = 19634

In [181]:
factions.loc[9711, 'home_system_id'] = 11979
factions.loc[9712, 'home_system_id'] = 3268
factions.loc[9713, 'home_system_id'] = 9081
factions.loc[9714, 'home_system_id'] = 341020
factions.loc[9715, 'home_system_id'] = 438891
#
factions.loc[9716, 'home_system_id'] = 909557
factions.loc[9717, 'home_system_id'] = 22988
factions.loc[9719, 'home_system_id'] = 232948
factions.loc[9720, 'home_system_id'] = 17620
factions.loc[9721, 'home_system_id'] = 86291
#
factions.loc[9722, 'home_system_id'] = 26060
factions.loc[9723, 'home_system_id'] = 22781
factions.loc[9724, 'home_system_id'] = 32174
factions.loc[9725, 'home_system_id'] = 57816
factions.loc[9726, 'home_system_id'] = 444381
#
factions.loc[9727, 'home_system_id'] = 25653
factions.loc[9729, 'home_system_id'] = 1463913
factions.loc[9730, 'home_system_id'] = 48382
factions.loc[9731, 'home_system_id'] = 615232
factions.loc[9732, 'home_system_id'] = 10059
#
factions.loc[9733, 'home_system_id'] = 48382
factions.loc[9734, 'home_system_id'] = 33689
factions.loc[9735, 'home_system_id'] = 25653
factions.loc[9736, 'home_system_id'] = 379577
factions.loc[9737, 'home_system_id'] = 172635

In [ ]:
factions.loc[9738, 'home_system_id'] = 187511
factions.loc[9739, 'home_system_id'] = 172635
factions.loc[9740, 'home_system_id'] = 35331
factions.loc[9741, 'home_system_id'] = 383016
factions.loc[9742, 'home_system_id'] = 379577
#
factions.loc[9743, 'home_system_id'] = 9269
factions.loc[9744, 'home_system_id'] = 32534
factions.loc[9745, 'home_system_id'] = 4689935
factions.loc[9747, 'home_system_id'] = 3570
factions.loc[9748, 'home_system_id'] = 19470
#
factions.loc[9750, 'home_system_id'] = 846
factions.loc[9752, 'home_system_id'] = 15976
factions.loc[9756, 'home_system_id'] = 2223
factions.loc[9757, 'home_system_id'] = 5459
factions.loc[9758, 'home_system_id'] = 19585
#
factions.loc[9759, 'home_system_id'] = 13566
factions.loc[9760, 'home_system_id'] = 341020
factions.loc[9762, 'home_system_id'] = 10931086
factions.loc[9765, 'home_system_id'] = 86291
factions.loc[9769, 'home_system_id'] = 12779
#
factions.loc[9770, 'home_system_id'] = 523
factions.loc[9771, 'home_system_id'] = 11620
factions.loc[9772, 'home_system_id'] = 4314
factions.loc[9774, 'home_system_id'] = 11614
factions.loc[9777, 'home_system_id'] = 1463913

In [240]:
factions.loc[9778, 'home_system_id'] = 15656
factions.loc[9779, 'home_system_id'] = 17620
factions.loc[9781, 'home_system_id'] = 19857
factions.loc[9786, 'home_system_id'] = 10309
factions.loc[9787, 'home_system_id'] = 12083
#
factions.loc[9788, 'home_system_id'] = 17118
factions.loc[9789, 'home_system_id'] = 7105
factions.loc[9790, 'home_system_id'] = 16047
factions.loc[9791, 'home_system_id'] = 13799
factions.loc[9792, 'home_system_id'] = 11557
#
factions.loc[9794, 'home_system_id'] = 15507
factions.loc[9795, 'home_system_id'] = 17743
factions.loc[9796, 'home_system_id'] = 14803
factions.loc[9798, 'home_system_id'] = 1439
factions.loc[9799, 'home_system_id'] = 57816
#
factions.loc[9800, 'home_system_id'] = 4756
factions.loc[9801, 'home_system_id'] = 2889
factions.loc[9803, 'home_system_id'] = 11802
factions.loc[9808, 'home_system_id'] = 13540
factions.loc[9810, 'home_system_id'] = 517
#
factions.loc[9811, 'home_system_id'] = 6936
factions.loc[9813, 'home_system_id'] = 15041
factions.loc[9814, 'home_system_id'] = 27167
factions.loc[9817, 'home_system_id'] = 11983
factions.loc[9818, 'home_system_id'] = 12653

In [ ]:
factions.loc[9819, 'home_system_id'] = 232948
factions.loc[9820, 'home_system_id'] = 16472
factions.loc[9821, 'home_system_id'] = 4138
factions.loc[9823, 'home_system_id'] = 13879
factions.loc[9825, 'home_system_id'] = 3347
#
factions.loc[9827, 'home_system_id'] = 8902
factions.loc[9829, 'home_system_id'] = 12139
factions.loc[9830, 'home_system_id'] = 11305
factions.loc[9831, 'home_system_id'] = 32534
factions.loc[9832, 'home_system_id'] = 32174
#
factions.loc[9833, 'home_system_id'] = 4689935
factions.loc[9834, 'home_system_id'] = 35331
factions.loc[9835, 'home_system_id'] = 3405
factions.loc[9836, 'home_system_id'] = 7640
factions.loc[9837, 'home_system_id'] = 14841
#
factions.loc[9838, 'home_system_id'] = 13858
factions.loc[9841, 'home_system_id'] = 19414
factions.loc[9842, 'home_system_id'] = 22
factions.loc[9844, 'home_system_id'] = 11276
factions.loc[9845, 'home_system_id'] = 4687
#
factions.loc[9849, 'home_system_id'] = 11142
factions.loc[9850, 'home_system_id'] = 13322
factions.loc[9851, 'home_system_id'] = 17679
factions.loc[9856, 'home_system_id'] = 18086
factions.loc[9857, 'home_system_id'] = 17685

In [384]:
factions.loc[9858, 'home_system_id'] = 15846
factions.loc[9859, 'home_system_id'] = 7610
factions.loc[9860, 'home_system_id'] = 500025
factions.loc[9861, 'home_system_id'] = 500025
factions.loc[9864, 'home_system_id'] = 55633
#
factions.loc[9865, 'home_system_id'] = 55633
factions.loc[9866, 'home_system_id'] = 2328
factions.loc[9869, 'home_system_id'] = 4859
factions.loc[9870, 'home_system_id'] = 19541
factions.loc[9877, 'home_system_id'] = 10918708
#
factions.loc[9884, 'home_system_id'] = 10930187
factions.loc[9887, 'home_system_id'] = 16663
factions.loc[9888, 'home_system_id'] = 45103
factions.loc[9889, 'home_system_id'] = 22431
factions.loc[9890, 'home_system_id'] = 45103
#
factions.loc[9922, 'home_system_id'] = 45103
factions.loc[9923, 'home_system_id'] = 1487905
factions.loc[9924, 'home_system_id'] = 27216
factions.loc[9925, 'home_system_id'] = 27216
factions.loc[9930, 'home_system_id'] = 17049
#
factions.loc[9933, 'home_system_id'] = 7652
factions.loc[9935, 'home_system_id'] = 1805
factions.loc[9938, 'home_system_id'] = 10849
factions.loc[9939, 'home_system_id'] = 6259676
factions.loc[9940, 'home_system_id'] = 16827

In [ ]:
factions.loc[9941, 'home_system_id'] = 5166139
factions.loc[9942, 'home_system_id'] = 5166139
factions.loc[9943, 'home_system_id'] = 5166139
factions.loc[9944, 'home_system_id'] = 5166139
#
factions.loc[9947, 'home_system_id'] = 13607222
factions.loc[9948, 'home_system_id'] = 13607222
factions.loc[9949, 'home_system_id'] = 13576538
factions.loc[9951, 'home_system_id'] = 13576538
#
factions.loc[9952, 'home_system_id'] = 3971
factions.loc[9953, 'home_system_id'] = 1266
factions.loc[9954, 'home_system_id'] = 19090
factions.loc[9955, 'home_system_id'] = 4419
factions.loc[9956, 'home_system_id'] = 7361
#
factions.loc[9957, 'home_system_id'] = 12899
factions.loc[9958, 'home_system_id'] = 14502
factions.loc[9960, 'home_system_id'] = 15640
factions.loc[9961, 'home_system_id'] = 33555
factions.loc[9962, 'home_system_id'] = 11530
#
factions.loc[9963, 'home_system_id'] = 8678
factions.loc[9965, 'home_system_id'] = 10834
factions.loc[9967, 'home_system_id'] = 4923
factions.loc[9968, 'home_system_id'] = 21962
factions.loc[9969, 'home_system_id'] = 15708

In [ ]:
factions.loc[9970, 'home_system_id'] = 19051
factions.loc[9971, 'home_system_id'] = 2871
factions.loc[9973, 'home_system_id'] = 14061
factions.loc[9974, 'home_system_id'] = 1793
factions.loc[9975, 'home_system_id'] = 13366
#
factions.loc[9976, 'home_system_id'] = 10441
factions.loc[9977, 'home_system_id'] = 5633
factions.loc[9978, 'home_system_id'] = 5683
factions.loc[9979, 'home_system_id'] = 17065
factions.loc[9980, 'home_system_id'] = 15897
#
factions.loc[9981, 'home_system_id'] = 3974
factions.loc[9982, 'home_system_id'] = 18834
factions.loc[9983, 'home_system_id'] = 6652
factions.loc[9984, 'home_system_id'] = 15166
factions.loc[9985, 'home_system_id'] = 16492
#
factions.loc[9986, 'home_system_id'] = 14598123
factions.loc[9987, 'home_system_id'] = 14598123
factions.loc[9988, 'home_system_id'] = 14598123
factions.loc[9991, 'home_system_id'] = 19267
factions.loc[9992, 'home_system_id'] = 13106
#
factions.loc[9996, 'home_system_id'] = 20566
factions.loc[9997, 'home_system_id'] = 16226
factions.loc[9998, 'home_system_id'] = 20464
factions.loc[9999, 'home_system_id'] = 50725
factions.loc[9606, 'home_system_id'] = 694257 #initial return mistaken, caught on double check

In [493]:
systems[systems['name'] == "Hyades Sector AB-W b2-2"]

,allegiance,allegiance_id,controlling_minor_faction,controlling_minor_faction_id,edsm_id,government,government_id,id,is_populated,minor_faction_presences,...,reserve_type_id,security,security_id,simbad_ref,state,state_id,updated_at,x,y,z
19949,Independent,4.0,Sirius Inc,7329.0,387421.0,Democracy,96.0,694257,True,"[{'minor_faction_id': 75107, 'state_id': 64, '...",...,1.0,Medium,32.0,,Boom,16.0,2018-02-22 03:12:17,-91.65625,-62.53125,-224.5


In [494]:
factions.loc[9606, 'home_system_id'] = 694257

In [497]:
missing_home = factions[factions['home_system_id'].isnull()].index
missing_home
#rest are either misspellings or empty

Int64Index([638, 3685, 9586, 9619, 9634, 9674], dtype='int64')

In [498]:
factions.drop(missing_home, inplace = True)

In [499]:
factions[factions['home_system_id'].isnull()]
#all the nulls are finally gone. Glory hallelujah.

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction


In [500]:
factions['home_system_id'] = factions['home_system_id'].astype(int)
factions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75787 entries, 0 to 75841
Data columns (total 11 columns):
id                   75787 non-null int64
name                 75787 non-null object
updated_at           75787 non-null int64
government_id        75787 non-null int64
government           75787 non-null object
allegiance_id        75787 non-null int64
allegiance           75787 non-null object
state_id             75787 non-null int64
state                75787 non-null object
home_system_id       75787 non-null int64
is_player_faction    75787 non-null int64
dtypes: int64(7), object(4)
memory usage: 6.9+ MB


In [501]:
factions.set_index('id', inplace = True)

In [503]:
#elite.upload(factions, 'factions')
#once done, doesn't need to be rerun

In [504]:
#factions.to_csv('clean_factions.csv')
#fun fact - I don't want to have to do all this cleaning again!

In [505]:
elite.query("""SELECT * FROM factions
            LIMIT 20""")

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
0,1,39 b Draconis One,1517130437,80,Cooperative,4,Independent,80,None,185,0
1,3,39 b Draconis Silver State Corp.,1509220271,64,Corporate,4,Independent,80,None,185,0
2,4,39 b Draconis Crimson Cartel,1514419230,16,Anarchy,4,Independent,80,None,185,0
3,6,33 Bootis Worker's Party,1519229966,32,Communism,4,Independent,80,None,156,0
4,99,Friends of Addo,1519564244,80,Cooperative,4,Independent,64,Civil War,456,0
5,100,Addo Silver Federal Holdings,1519153494,64,Corporate,3,Federation,16,Boom,456,0
6,101,Addo Dominion,1519662844,128,Feudal,2,Empire,16,Boom,456,0
7,102,Atroco Emperor's Grace,1519278181,144,Patronage,2,Empire,16,Boom,1455,0
8,103,Addo Brothers,1519564244,16,Anarchy,4,Independent,64,Civil War,456,0
9,105,Addo Silver Advanced Partners,1519245119,64,Corporate,3,Federation,73,War,456,0


In [7]:
has_listings = listings['station_id'].unique()

In [49]:
station_drop = []
for i in stations.index:
    if stations.loc[i, 'id'] not in has_listings:
        station_drop.append(i)
mkt_stations = stations.drop(station_drop)

In [42]:
mkt_stations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47596 entries, 0 to 67703
Data columns (total 39 columns):
allegiance                      47441 non-null object
allegiance_id                   47441 non-null float64
body_id                         13821 non-null float64
controlling_minor_faction_id    47441 non-null float64
distance_to_star                47233 non-null float64
economies                       47596 non-null object
export_commodities              47596 non-null object
government                      47441 non-null object
government_id                   47441 non-null float64
has_blackmarket                 47596 non-null bool
has_commodities                 47596 non-null bool
has_docking                     47596 non-null bool
has_market                      47596 non-null bool
has_outfitting                  47596 non-null bool
has_rearm                       47596 non-null bool
has_refuel                      47596 non-null bool
has_repair                      4759

In [10]:
mkt_stations[mkt_stations['distance_to_star'].isnull() & (mkt_stations['body_id'].isnull() != 1)]

,allegiance,allegiance_id,body_id,controlling_minor_faction_id,distance_to_star,economies,export_commodities,government,government_id,has_blackmarket,...,settlement_security_id,settlement_size,settlement_size_id,shipyard_updated_at,state,state_id,system_id,type,type_id,updated_at
5886,None,NaN,518407.0,NaN,NaN,[Refinery],[],None,NaN,False,...,NaN,None,NaN,NaT,None,NaN,777,Planetary Outpost,13.0,2017-09-22 22:32:05
7384,Empire,2.0,416349.0,18907.0,NaN,[Refinery],[],Patronage,144.0,False,...,NaN,None,NaN,NaT,Boom,16.0,1065,Planetary Outpost,13.0,2018-02-16 17:26:54
7523,Independent,4.0,1658354.0,25677.0,NaN,[Refinery],[],Feudal,128.0,False,...,NaN,None,NaN,NaT,None,80.0,1685,Planetary Outpost,13.0,2018-02-19 21:58:31
8149,Independent,4.0,2638.0,56689.0,NaN,[Industrial],[],Corporate,64.0,False,...,NaN,None,NaN,NaT,Civil War,64.0,1891,Planetary Outpost,13.0,2018-02-13 02:05:28
8202,Federation,3.0,2907551.0,59192.0,NaN,[Refinery],[],Democracy,96.0,True,...,NaN,None,NaN,NaT,None,80.0,554,Planetary Outpost,13.0,2017-12-23 17:03:29
27675,Federation,3.0,1972225.0,6655.0,NaN,[Colony],[],Democracy,96.0,True,...,NaN,None,NaN,NaT,None,80.0,8118,Planetary Outpost,13.0,2017-11-26 20:31:39
28905,Independent,4.0,897674.0,47126.0,NaN,[Colony],[],Cooperative,80.0,False,...,NaN,None,NaN,NaT,Famine,37.0,7728,Planetary Outpost,13.0,2018-02-18 17:53:21
29169,Independent,4.0,1117792.0,59877.0,NaN,[Military],[],Dictatorship,112.0,False,...,NaN,None,NaN,NaT,None,80.0,5920,Planetary Port,14.0,2018-02-06 17:14:33
29659,Independent,4.0,1552589.0,74725.0,NaN,[Colony],[],Cooperative,80.0,False,...,NaN,None,NaN,NaT,None,80.0,7509,Planetary Outpost,13.0,2017-07-14 03:59:31
37725,Federation,3.0,3628193.0,63814.0,NaN,[Military],[],Corporate,64.0,False,...,NaN,None,NaN,2018-01-27 19:49:16,None,80.0,11062,Planetary Port,14.0,2018-02-16 02:49:01


In [11]:
bodies = pd.read_json('./bodies_recently.jsonl', lines=True)

In [12]:
known_body = []
for bdy in mkt_stations[mkt_stations['distance_to_star'].isnull() & (mkt_stations['body_id'].isnull() != 1)]['body_id']:
    if bdy in bodies['id'].unique():
        known_body.append(bdy)
known_body

[1552589.0]

In [13]:
bodies[bodies['id'] == 1552589]

,age,arg_of_periapsis,atmosphere_composition,atmosphere_type_id,atmosphere_type_name,axis_tilt,belt_moon_masses,catalogue_gliese_id,catalogue_hd_id,catalogue_hipp_id,...,surface_pressure,surface_temperature,system_id,terraforming_state_id,terraforming_state_name,type_id,type_name,updated_at,volcanism_type_id,volcanism_type_name
13414,NaN,262.192993,[],1.0,No atmosphere,-0.425017,NaN,None,None,None,...,0.0,183.6064,7509,1.0,Not terraformable,33.0,Rocky body,2018-04-02 09:48:20,4.0,Iron magma


In [14]:
bodies.loc[13414]['distance_to_arrival']

806.0

In [15]:
mkt_stations[mkt_stations['body_id'] == 1552589]

,allegiance,allegiance_id,body_id,controlling_minor_faction_id,distance_to_star,economies,export_commodities,government,government_id,has_blackmarket,...,settlement_security_id,settlement_size,settlement_size_id,shipyard_updated_at,state,state_id,system_id,type,type_id,updated_at
29659,Independent,4.0,1552589.0,74725.0,NaN,[Colony],[],Cooperative,80.0,False,...,NaN,None,NaN,NaT,None,80.0,7509,Planetary Outpost,13.0,2017-07-14 03:59:31


In [50]:
mkt_stations.loc[29659, 'distance_to_star'] = bodies.loc[13414]['distance_to_arrival']

In [51]:
mkt_stations[(mkt_stations['distance_to_star'].isnull() != 1) & (mkt_stations['body_id'].isnull() != 1)]

,allegiance,allegiance_id,body_id,controlling_minor_faction_id,distance_to_star,economies,export_commodities,government,government_id,has_blackmarket,...,settlement_security_id,settlement_size,settlement_size_id,shipyard_updated_at,state,state_id,system_id,type,type_id,updated_at
0,Federation,3.0,7086578.0,13925.0,171.0,[Agriculture],"[Mineral Oil, Fruit and Vegetables, Grain]",Corporate,64.0,True,...,NaN,None,NaN,2018-02-25 18:17:16,Civil War,64.0,396,Orbis Starport,8.0,2018-02-24 20:39:41
1,Federation,3.0,3433854.0,39484.0,1080.0,"[High Tech, Refinery]","[Hydrogen Fuel, Aluminium, Advanced Catalysers]",Democracy,96.0,False,...,NaN,None,NaN,2018-02-26 02:54:11,None,80.0,773,Coriolis Starport,3.0,2018-02-26 02:54:05
6,Federation,3.0,6938.0,13968.0,994.0,[Service],"[Hydrogen Fuel, Biowaste, Limpet]",Corporate,64.0,True,...,NaN,None,NaN,2018-02-26 00:39:12,None,80.0,189,Orbis Starport,8.0,2018-02-24 22:25:45
7,Empire,2.0,7885.0,9055.0,67.0,"[High Tech, Refinery]","[Hydrogen Fuel, Aluminium, Animal Monitors]",Patronage,144.0,False,...,NaN,None,NaN,2018-02-26 03:09:14,Retreat,96.0,526,Coriolis Starport,3.0,2018-02-20 13:22:33
9,Independent,4.0,9940.0,18979.0,1759.0,"[Industrial, Refinery]","[Food Cartridges, Water Purifiers, Building Fa...",Cooperative,80.0,True,...,NaN,None,NaN,2018-02-25 15:43:13,Boom,16.0,1120,Coriolis Starport,3.0,2018-02-26 00:26:13
11,Independent,4.0,8095426.0,18444.0,582.0,[Agriculture],[],Corporate,64.0,True,...,NaN,None,NaN,2018-02-08 19:43:41,None,80.0,345,Orbis Starport,8.0,2018-02-21 23:21:57
18,Alliance,1.0,7279164.0,1082.0,13357.0,[Agriculture],"[Algae, Grain]",Democracy,96.0,True,...,NaN,None,NaN,2018-02-25 22:08:27,None,80.0,327,Orbis Starport,8.0,2018-02-25 19:24:58
21,Independent,4.0,8095381.0,18444.0,582.0,[Agriculture],[],Corporate,64.0,True,...,NaN,None,NaN,2018-02-25 18:13:48,None,80.0,345,Orbis Starport,8.0,2018-02-25 18:13:42
23,Federation,3.0,2766442.0,33282.0,234.0,"[Industrial, Refinery]","[Food Cartridges, Water Purifiers, Building Fa...",Corporate,64.0,True,...,NaN,None,NaN,2018-02-25 17:18:13,None,80.0,208,Coriolis Starport,3.0,2018-02-24 15:59:43
24,Independent,4.0,2502202.0,55037.0,111.0,[High Tech],"[Hydrogen Fuel, Pesticides, Animal Monitors]",Theocracy,160.0,True,...,NaN,None,NaN,2018-02-26 03:01:26,None,80.0,1488,Orbis Starport,8.0,2018-02-26 03:23:56


In [43]:
unk_bdy_list = mkt_stations[(mkt_stations['distance_to_star'].isnull()) & (mkt_stations['body_id'].isnull() != 1)]['body_id']
unk_bdy_list

5886       518407.0
7384       416349.0
7523      1658354.0
8149         2638.0
8202      2907551.0
27675     1972225.0
28905      897674.0
29169     1117792.0
29659     1552589.0
37725     3628193.0
38426      314679.0
38707       78470.0
48308     1132931.0
48976     3151459.0
56546    13222990.0
57897     2483509.0
58000     2873422.0
58267    10060186.0
59241     2176280.0
64859     3058584.0
65718     2922893.0
65742     1119426.0
65809      903537.0
65822     1836783.0
65862     1273432.0
65901     1246543.0
65966     3367845.0
65978     1246569.0
66010     1447206.0
66145     2096675.0
66188     2410882.0
66267     2870260.0
66321     3259994.0
66348     3647140.0
Name: body_id, dtype: float64

In [53]:
k_bdy_list = mkt_stations[(mkt_stations['distance_to_star'].isnull() != 1) & (mkt_stations['body_id'].isnull() != 1)]['body_id']
k_bdy_list

0         7086578.0
1         3433854.0
6            6938.0
7            7885.0
9            9940.0
11        8095426.0
18        7279164.0
21        8095381.0
23        2766442.0
24        2502202.0
31         167444.0
33        1937650.0
36           7409.0
43        2076826.0
50        2755590.0
51         102653.0
53        2755776.0
54        7278989.0
55          14229.0
59         167375.0
62         379934.0
67          10267.0
72        7278978.0
73        2755689.0
76        1937593.0
79         117694.0
80        7279193.0
82          17024.0
88          17026.0
99          16811.0
            ...    
67474     9879109.0
67475    12559241.0
67489    15001340.0
67495    16001939.0
67509    11222662.0
67510     6787734.0
67514    13362898.0
67516    14532081.0
67523     6532328.0
67533    13376390.0
67539    13525877.0
67548    11437547.0
67552     3702678.0
67554    12842327.0
67561    10209295.0
67566    11426024.0
67573    12106653.0
67574     2127568.0
67584    13995970.0


In [54]:
for ubl in unk_bdy_list:
    if ubl in k_bdy_list:
        print(ubl)

In [55]:
mkt_stations[mkt_stations['distance_to_star'].isnull()]['system_id']

5886         777
6008        1367
6089        1200
6090        1200
6091        1200
6092        1200
6107         732
6108         732
6197        1578
6232        1096
6565        1098
6916         861
7088        1565
7192        1460
7227        2032
7384        1065
7523        1685
7570         861
7572         861
7609         896
7642        1521
7720         543
7853        1470
7893        1926
7904        1598
7958        1553
7974        1820
7999        2290
8019        1665
8036        2352
          ...   
65957      61947
65964     382417
65966     240650
65978     139073
66010     289552
66016    3850105
66020     598381
66113    3941183
66145     141704
66163     513319
66188    4029050
66210     517456
66235    1852691
66236    1454506
66245     960174
66262     120370
66267      66745
66286      80297
66300    3550047
66312      86546
66321      53883
66324      62008
66335    5668224
66338      47267
66348    3490374
66412     708823
66510      43831
66665     4524

In [56]:
#there's no real way to impute without manually travelling to the systems, and ultimately distance
#will be a crucial component of calculating efficiency. I can't just ignore it here. Though I would rather
#not have to do so, I'll have to drop listings from stations that I don't have distances for. Shouldn't narrow
#the database too much, but I don't really have any other option for this to work
missing_dist = mkt_stations[mkt_stations['distance_to_star'].isnull()]['id']

In [24]:
drop_listings = []
for i in new_listings.index:
    if new_listings.loc[i, 'station_id'] in missing_dist:
        drop_listings.append(i)
len(drop_listings)

15958

In [25]:
new_listings.drop(drop_listings, inplace=True)

In [26]:
new_listings.shape

(3171411, 7)

In [27]:
new_listings.set_index('id', inplace=True)

In [87]:
#elite.insert("DROP TABLE listings;")

In [88]:
#elite.upload(new_listings, 'listings')

In [89]:
elite.query("""SELECT * FROM listings
               LIMIT 20""")

,id,station_id,commodity_id,supply,buy_price,sell_price,demand
0,1,1,5,0,0,744,1159
1,2,1,6,0,0,7474,38
2,3,1,7,0,0,936,355
3,12,1,8,0,0,913,12054
4,13,1,9,0,0,1431,2121
5,14,1,12,0,0,262,287
6,15,1,30,0,0,936,269
7,16,1,31,0,0,711,440
8,17,1,33,0,0,2026,4136
9,18,1,36,0,0,7474,11


In [57]:
mkt_stations.dropna(subset=['distance_to_star'], inplace=True)

In [58]:
mkt_stations['distance_to_star'] = mkt_stations['distance_to_star'].astype(int)

In [59]:
mkt_stations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47234 entries, 0 to 67703
Data columns (total 39 columns):
allegiance                      47104 non-null object
allegiance_id                   47104 non-null float64
body_id                         13788 non-null float64
controlling_minor_faction_id    47104 non-null float64
distance_to_star                47234 non-null int64
economies                       47234 non-null object
export_commodities              47234 non-null object
government                      47104 non-null object
government_id                   47104 non-null float64
has_blackmarket                 47234 non-null bool
has_commodities                 47234 non-null bool
has_docking                     47234 non-null bool
has_market                      47234 non-null bool
has_outfitting                  47234 non-null bool
has_rearm                       47234 non-null bool
has_refuel                      47234 non-null bool
has_repair                      47234 

In [60]:
#these values are almost all nulls, and aren't valuable to me anyway, so
mkt_stations.drop(['settlement_security', 'settlement_security_id', 'settlement_size', 'settlement_size_id'], axis=1, inplace=True)

In [32]:
mkt_stations[mkt_stations['type'].isnull()]['name']

5345     Evans Settlement
9899     Tiliala's Lament
66727     Foster Terminal
Name: name, dtype: object

In [117]:
mkt_stations.loc[66727]['system_id']

10918178

In [118]:
systems[systems['id'] == 10918178]['name']

20452    Coeus
Name: name, dtype: object

In [61]:
mkt_stations.loc[5345, 'type'] = 'Coriolis Starport'
mkt_stations.loc[5345, 'type_id'] = 3

In [62]:
mkt_stations.loc[9899, 'type'] = 'Megaship'
mkt_stations.loc[9899, 'type_id'] = 19

In [63]:
mkt_stations.loc[66727, 'type'] = 'Megaship'
mkt_stations.loc[66727, 'type_id'] = 19

In [107]:
mkt_stations['type'].unique()

array(['Orbis Starport', 'Coriolis Starport', 'Civilian Outpost',
       'Industrial Outpost', 'Ocellus Starport', 'Commercial Outpost',
       'Mining Outpost', 'Scientific Outpost', 'Military Outpost',
       'Asteroid Base', 'Unknown Starport', None, 'Planetary Outpost',
       'Planetary Port', 'Unknown Planetary', 'Planetary Engineer Base',
       'Megaship', 'Unknown Outpost'], dtype=object)

In [64]:
mkt_stations['type_id'] = mkt_stations['type_id'].astype(int)

In [65]:
mkt_stations[mkt_stations['state'].isnull()]

,allegiance,allegiance_id,body_id,controlling_minor_faction_id,distance_to_star,economies,export_commodities,government,government_id,has_blackmarket,...,prohibited_commodities,selling_modules,selling_ships,shipyard_updated_at,state,state_id,system_id,type,type_id,updated_at
1512,None,NaN,NaN,NaN,1186,[Industrial],[],None,NaN,True,...,[],"[738, 739, 740, 741, 742, 748, 749, 750, 751, ...",[],NaT,None,NaN,564,Commercial Outpost,2,2018-02-24 11:19:35
1521,None,NaN,NaN,NaN,15925,"[Extraction, Refinery]","[Gold, Silver, Gallite]",None,NaN,False,...,"[Narcotics, Combat Stabilisers, Slaves, Battle...",[],[],NaT,None,NaN,1616,Industrial Outpost,4,2017-09-19 02:14:53
1863,None,NaN,NaN,NaN,27,"[Extraction, Industrial]",[],None,NaN,False,...,[],[],[],NaT,None,NaN,953,Industrial Outpost,4,2018-01-15 00:00:36
1912,None,NaN,5469097.0,NaN,6368,"[Extraction, Refinery]",[],None,NaN,True,...,"[Beer, Liquor, Narcotics, Tobacco, Wine, Comba...","[738, 739, 740, 741, 742, 743, 744, 745, 746, ...",[],NaT,None,NaN,371,Civilian Outpost,1,2018-02-16 17:54:35
2295,None,NaN,NaN,NaN,16,"[Extraction, Industrial]",[],None,NaN,False,...,[],"[738, 739, 740, 741, 742, 743, 744, 745, 746, ...",[],NaT,None,NaN,953,Civilian Outpost,1,2018-01-15 00:00:36
2453,None,NaN,NaN,NaN,617,"[Extraction, Refinery]","[Aluminium, Gallite, Lepidolite]",None,NaN,False,...,"[Narcotics, Tobacco, Combat Stabilisers, Slave...","[738, 739, 740, 741, 742, 743, 744, 745, 746, ...",[],NaT,None,NaN,945,Mining Outpost,6,2018-01-14 17:14:45
2454,None,NaN,NaN,NaN,846,"[Extraction, Refinery]","[Copper, Titanium, Lepidolite]",None,NaN,True,...,"[Narcotics, Combat Stabilisers, Imperial Slave...","[738, 739, 740, 741, 742, 748, 749, 750, 751, ...",[],NaT,None,NaN,945,Civilian Outpost,1,2018-01-04 08:51:00
4638,None,NaN,NaN,NaN,85,"[Industrial, Refinery]",[],None,NaN,False,...,[],[],[],NaT,None,NaN,658,Industrial Outpost,4,2017-12-01 04:28:15
6167,None,NaN,11657034.0,NaN,5569,[Military],"[Hydrogen Fuel, Scrap, Limpet]",None,NaN,False,...,"[Slaves, Toxic Waste]","[758, 759, 760, 761, 762, 880, 881, 890, 892, ...",[],NaT,None,NaN,2302,Planetary Outpost,13,2018-02-23 20:32:55
6767,None,NaN,NaN,NaN,228,[Colony],[],None,NaN,False,...,[],"[758, 759, 760, 761, 762, 763, 764, 765, 766, ...",[],NaT,None,NaN,620,Planetary Outpost,13,2017-11-02 10:47:54


In [66]:
mkt_stations[mkt_stations['state'].isnull() != 1].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47104 entries, 0 to 67703
Data columns (total 35 columns):
allegiance                      47104 non-null object
allegiance_id                   47104 non-null float64
body_id                         13737 non-null float64
controlling_minor_faction_id    47104 non-null float64
distance_to_star                47104 non-null int64
economies                       47104 non-null object
export_commodities              47104 non-null object
government                      47104 non-null object
government_id                   47104 non-null float64
has_blackmarket                 47104 non-null bool
has_commodities                 47104 non-null bool
has_docking                     47104 non-null bool
has_market                      47104 non-null bool
has_outfitting                  47104 non-null bool
has_rearm                       47104 non-null bool
has_refuel                      47104 non-null bool
has_repair                      47104 

In [76]:
#nulls are all common, and probably has something to do with lacking the proper faction information
#because we have the system information and that system has faction info, we should be able to impute it
new_factions = pd.read_csv('./clean_factions.csv')
for i in mkt_stations[mkt_stations['state'].isnull()].index:
    sys_id = mkt_stations.loc[i, 'system_id']
    mkt_stations.loc[i, 'controlling_minor_faction_id'] = list(systems[systems['id'] == sys]['controlling_minor_faction_id'])[0]

In [77]:
#allegiance, etc, can all be dropped, as they're all identical to the faction's, meaning we don't need second copies
mkt_stations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47234 entries, 0 to 67703
Data columns (total 35 columns):
allegiance                      47104 non-null object
allegiance_id                   47104 non-null float64
body_id                         13788 non-null float64
controlling_minor_faction_id    47234 non-null float64
distance_to_star                47234 non-null int64
economies                       47234 non-null object
export_commodities              47234 non-null object
government                      47104 non-null object
government_id                   47104 non-null float64
has_blackmarket                 47234 non-null bool
has_commodities                 47234 non-null bool
has_docking                     47234 non-null bool
has_market                      47234 non-null bool
has_outfitting                  47234 non-null bool
has_rearm                       47234 non-null bool
has_refuel                      47234 non-null bool
has_repair                      47234 

In [78]:
mkt_stations.drop(['allegiance', 'allegiance_id', 'government', 'government_id', 'state', 'state_id'], axis=1, inplace=True)

In [ ]:
#we don't need the body ids, as that information is scatter-shot (and the full body database is too gargantuan
#to reasonably incorporate, even if the source gathering that didn't stop aggregating that some time ago)
mkt_stations.drop('body_id', axis=1, inplace=True)

In [82]:
mkt_stations['controlling_minor_faction_id'] = mkt_stations['controlling_minor_faction_id'].astype(int)

In [83]:
mkt_stations.drop(['market_updated_at', 'outfitting_updated_at', 'shipyard_updated_at', 'updated_at'], axis=1, inplace=True)

In [84]:
mkt_stations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47234 entries, 0 to 67703
Data columns (total 24 columns):
controlling_minor_faction_id    47234 non-null int64
distance_to_star                47234 non-null int64
economies                       47234 non-null object
export_commodities              47234 non-null object
has_blackmarket                 47234 non-null bool
has_commodities                 47234 non-null bool
has_docking                     47234 non-null bool
has_market                      47234 non-null bool
has_outfitting                  47234 non-null bool
has_rearm                       47234 non-null bool
has_refuel                      47234 non-null bool
has_repair                      47234 non-null bool
has_shipyard                    47234 non-null bool
id                              47234 non-null int64
import_commodities              47234 non-null object
is_planetary                    47234 non-null bool
max_landing_pad_size            47231 non-null obj

In [86]:
mkt_stations[mkt_stations['max_landing_pad_size'].isnull()]['name']

9999     Rescue Ship - Armstrong Enterprise
19982                            Malkendorf
49999          Rescue Ship - Beaufoy Vision
Name: name, dtype: object

In [89]:
mkt_stations[mkt_stations['max_landing_pad_size'].isnull()]['type']

9999     Megaship
19982    Megaship
49999    Megaship
Name: type, dtype: object

In [90]:
mkt_stations[mkt_stations['type'] == 'Megaship']['max_landing_pad_size']

9899        L
9999     None
19982    None
39802       L
49999    None
66295       L
66296       L
66592       L
66644       L
66727       L
66766       L
67672       L
67679       L
Name: max_landing_pad_size, dtype: object

In [91]:
mkt_stations.loc[9999, 'max_landing_pad_size'] = 'L'
mkt_stations.loc[19982, 'max_landing_pad_size'] = 'L'
mkt_stations.loc[49999, 'max_landing_pad_size'] = 'L'

In [93]:
mkt_stations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47234 entries, 0 to 67703
Data columns (total 24 columns):
controlling_minor_faction_id    47234 non-null int64
distance_to_star                47234 non-null int64
economies                       47234 non-null object
export_commodities              47234 non-null object
has_blackmarket                 47234 non-null bool
has_commodities                 47234 non-null bool
has_docking                     47234 non-null bool
has_market                      47234 non-null bool
has_outfitting                  47234 non-null bool
has_rearm                       47234 non-null bool
has_refuel                      47234 non-null bool
has_repair                      47234 non-null bool
has_shipyard                    47234 non-null bool
id                              47234 non-null int64
import_commodities              47234 non-null object
is_planetary                    47234 non-null bool
max_landing_pad_size            47234 non-null obj

In [94]:
mkt_stations.head()

,controlling_minor_faction_id,distance_to_star,economies,export_commodities,has_blackmarket,has_commodities,has_docking,has_market,has_outfitting,has_rearm,...,import_commodities,is_planetary,max_landing_pad_size,name,prohibited_commodities,selling_modules,selling_ships,system_id,type,type_id
0,13925,171,[Agriculture],"[Mineral Oil, Fruit and Vegetables, Grain]",True,True,True,True,True,True,...,"[Pesticides, Aquaponic Systems, Biowaste]",False,L,Reilly Hub,"[Narcotics, Tobacco, Combat Stabilisers, Imper...","[738, 739, 740, 743, 744, 745, 748, 749, 750, ...","[Adder, Eagle Mk. II, Hauler, Sidewinder Mk. I...",396,Orbis Starport,8
1,39484,1080,"[High Tech, Refinery]","[Hydrogen Fuel, Aluminium, Advanced Catalysers]",False,True,True,True,True,True,...,"[Cobalt, Rutile, Pyrophyllite]",False,L,Bounds Hub,"[Narcotics, Tobacco, Combat Stabilisers, Imper...","[738, 739, 740, 741, 742, 748, 749, 750, 751, ...","[Adder, Cobra Mk. III, Diamondback Scout, Fede...",773,Coriolis Starport,3
2,10957,1834,[Refinery],"[Aluminium, Copper, Titanium]",False,True,True,True,False,False,...,"[Microbial Furnaces, Bauxite, Lepidolite]",False,M,Starzl Plant,"[Slaves, Toxic Waste]",[],[],981,Civilian Outpost,1
3,33245,228,[Refinery],"[Cobalt, Gold, Silver]",False,True,True,True,True,True,...,"[Bertrandite, Gallite, Indite]",False,L,Lunan Hub,"[Narcotics, Tobacco, Combat Stabilisers, Imper...","[738, 739, 740, 741, 742, 748, 749, 750, 751, ...","[Hauler, Sidewinder Mk. I, Type-6 Transporter,...",2236,Orbis Starport,8
4,33245,306,[Refinery],"[Polymers, Copper, Titanium]",False,True,True,True,True,True,...,"[Microbial Furnaces, Bauxite, Lepidolite]",False,L,Hutchinson City,"[Narcotics, Tobacco, Combat Stabilisers, Imper...","[738, 739, 740, 741, 742, 748, 749, 750, 751, ...","[Hauler, Sidewinder Mk. I, Type-6 Transporter,...",2236,Orbis Starport,8


In [95]:
mkt_stations.loc[0]

controlling_minor_faction_id                                                13925
distance_to_star                                                              171
economies                                                           [Agriculture]
export_commodities                     [Mineral Oil, Fruit and Vegetables, Grain]
has_blackmarket                                                              True
has_commodities                                                              True
has_docking                                                                  True
has_market                                                                   True
has_outfitting                                                               True
has_rearm                                                                    True
has_refuel                                                                   True
has_repair                                                                   True
has_shipyard    

In [97]:
#we can get rid of import and export commodities - we have the numbers to figure that out ourselves.
#we're not incorporating modules and ships into this right now, so we can drop those two as well
mkt_stations.drop(['export_commodities', 'import_commodities', 'selling_modules', 'selling_ships'], axis=1, inplace=True)

In [99]:
mkt_stations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47234 entries, 0 to 67703
Data columns (total 20 columns):
controlling_minor_faction_id    47234 non-null int64
distance_to_star                47234 non-null int64
economies                       47234 non-null object
has_blackmarket                 47234 non-null bool
has_commodities                 47234 non-null bool
has_docking                     47234 non-null bool
has_market                      47234 non-null bool
has_outfitting                  47234 non-null bool
has_rearm                       47234 non-null bool
has_refuel                      47234 non-null bool
has_repair                      47234 non-null bool
has_shipyard                    47234 non-null bool
id                              47234 non-null int64
is_planetary                    47234 non-null bool
max_landing_pad_size            47234 non-null object
name                            47234 non-null object
prohibited_commodities          47234 non-null obj

In [110]:
#economies are important, as are whether a commodity is illegal or not. but the commodity seems like it'd be
#better suited for a listing, so let's take a look at listings from a single station
mkt_stations.loc[0]['id']

5

In [115]:
elite.query("""SELECT c.name, c.category, station_id, supply, buy_price, sell_price, demand FROM listings
               INNER JOIN commodities c ON commodity_id = c.id
               WHERE station_id = """ + str(mkt_stations.loc[0]['id']))

,name,category,station_id,supply,buy_price,sell_price,demand
0,Hydrogen Fuel,Chemicals,5,76991,104,98,0
1,Mineral Oil,Chemicals,5,86610,197,166,0
2,Pesticides,Chemicals,5,0,0,397,149132
3,Clothing,Consumer Items,5,0,0,696,281582
4,Consumer Technology,Consumer Items,5,0,0,7646,19299
5,Domestic Appliances,Consumer Items,5,0,0,1028,53938
6,Wine,Legal Drugs,5,17936,276,245,0
7,Atmospheric Processors,Machinery,5,0,0,853,9484
8,Crop Harvesters,Machinery,5,0,0,2882,184209
9,Marine Equipment,Machinery,5,0,0,5035,48562


In [116]:
mkt_stations.loc[0]['prohibited_commodities']

list

In [125]:
#nothing in the preliminary search, but we'll built a nice big long function to tell us whether that comes up
#illegal_listing = 0
#index_number = 0
#for i in mkt_stations.index:
#    for commod in elite.query("""SELECT c.name FROM listings
#                                INNER JOIN commodities c ON commodity_id = c.id
#                                WHERE station_id =""" + str(mkt_stations.loc[i, 'id']))['name']:
#        if commod in mkt_stations.loc[i, 'prohibited_commodities']:
#            illegal_listing += 1
#    index_number += 1
#    if index_number % 2000 == 0:
#        print(str(index_number) + " markets checked")
#illegal_listing
#this ended up taking way too much time - I think I'll separate the dictionary, then check after the function runs
#to check for illegalities

In [126]:
illegal_listings = mkt_stations['prohibited_commodities']
mkt_stations.drop('prohibited_commodities', axis=1, inplace=True)

In [130]:
econ_types = set()
for i in mkt_stations.index:
    for econ in mkt_stations.loc[i, 'economies']:
        econ_types.add(econ)

In [131]:
econ_types

{'Agriculture',
 'Colony',
 'Damaged',
 'Extraction',
 'High Tech',
 'Industrial',
 'Military',
 'Refinery',
 'Repair',
 'Rescue',
 'Service',
 'Terraforming',
 'Tourism'}

In [143]:
for etyp in econ_types:
    mkt_stations['econ_' + etyp.lower()] = False
    for i in mkt_stations.index:
        econ = mkt_stations.loc[i, 'economies']
        if etyp in econ:
            mkt_stations.loc[i, ('econ_' + etyp.lower())] = True
mkt_stations.head()

,controlling_minor_faction_id,distance_to_star,economies,has_blackmarket,has_commodities,has_docking,has_market,has_outfitting,has_rearm,has_refuel,...,econ_refinery,econ_military,econ_industrial,econ_colony,econ_service,econ_rescue,econ_agriculture,econ_terraforming,econ_high tech,econ_tourism
0,13925,171,[Agriculture],True,True,True,True,True,True,True,...,False,False,False,False,False,False,True,False,False,False
1,39484,1080,"[High Tech, Refinery]",False,True,True,True,True,True,True,...,True,False,False,False,False,False,False,False,True,False
2,10957,1834,[Refinery],False,True,True,True,False,False,True,...,True,False,False,False,False,False,False,False,False,False
3,33245,228,[Refinery],False,True,True,True,True,True,True,...,True,False,False,False,False,False,False,False,False,False
4,33245,306,[Refinery],False,True,True,True,True,True,True,...,True,False,False,False,False,False,False,False,False,False


In [139]:
mkt_stations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47234 entries, 0 to 67703
Data columns (total 32 columns):
controlling_minor_faction_id    47234 non-null int64
distance_to_star                47234 non-null int64
economies                       47234 non-null object
has_blackmarket                 47234 non-null bool
has_commodities                 47234 non-null bool
has_docking                     47234 non-null bool
has_market                      47234 non-null bool
has_outfitting                  47234 non-null bool
has_rearm                       47234 non-null bool
has_refuel                      47234 non-null bool
has_repair                      47234 non-null bool
has_shipyard                    47234 non-null bool
id                              47234 non-null int64
is_planetary                    47234 non-null bool
max_landing_pad_size            47234 non-null object
name                            47234 non-null object
system_id                       47234 non-null int

In [144]:
mkt_stations.head()['economies']

0            [Agriculture]
1    [High Tech, Refinery]
2               [Refinery]
3               [Refinery]
4               [Refinery]
Name: economies, dtype: object

In [145]:
mkt_stations.head()['econ_refinery']

0    False
1     True
2     True
3     True
4     True
Name: econ_refinery, dtype: bool

In [148]:
mkt_stations.head()['econ_high tech']

0    False
1     True
2    False
3    False
4    False
Name: econ_high tech, dtype: bool

In [149]:
mkt_stations.rename(columns={'econ_high tech':'econ_high_tech'}, inplace=True)

In [151]:
#with economies dummied out, we can drop the rest
mkt_stations.drop('economies', axis=1, inplace=True)

In [152]:
mkt_stations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47234 entries, 0 to 67703
Data columns (total 31 columns):
controlling_minor_faction_id    47234 non-null int64
distance_to_star                47234 non-null int64
has_blackmarket                 47234 non-null bool
has_commodities                 47234 non-null bool
has_docking                     47234 non-null bool
has_market                      47234 non-null bool
has_outfitting                  47234 non-null bool
has_rearm                       47234 non-null bool
has_refuel                      47234 non-null bool
has_repair                      47234 non-null bool
has_shipyard                    47234 non-null bool
id                              47234 non-null int64
is_planetary                    47234 non-null bool
max_landing_pad_size            47234 non-null object
name                            47234 non-null object
system_id                       47234 non-null int64
type                            47234 non-null obje

In [153]:
mkt_stations.set_index('id', inplace=True)

In [158]:
mkt_stations['has_commodities'].unique()

array([ True, False])

In [159]:
mkt_stations['has_docking'].unique()

array([ True])

In [157]:
mkt_stations['has_market'].unique()

array([ True])

In [162]:
mkt_stations['has_refuel'].unique()

array([ True, False])

In [163]:
#a couple of these are unnecessary (obviously they all have markets, all of them are available to be docked
#at), but apparently not all have refueling stationsand I presume some of these you can only sell at 
#(and not buy from). The others I would expect to be varied true/false and don't really need to double check
mkt_stations.drop(['has_docking', 'has_market'], axis=1, inplace=True)

In [164]:
elite.upload(mkt_stations, 'stations')
#once done doesn't need to be redone

In [165]:
elite.query("""SELECT * FROM stations
               LIMIT 20""")

,id,controlling_minor_faction_id,distance_to_star,has_blackmarket,has_commodities,has_outfitting,has_rearm,has_refuel,has_repair,has_shipyard,...,econ_refinery,econ_military,econ_industrial,econ_colony,econ_service,econ_rescue,econ_agriculture,econ_terraforming,econ_high_tech,econ_tourism
0,5,13925,171,True,True,True,True,True,True,True,...,False,False,False,False,False,False,True,False,False,False
1,14,39484,1080,False,True,True,True,True,True,True,...,True,False,False,False,False,False,False,False,True,False
2,28,10957,1834,False,True,False,False,True,False,False,...,True,False,False,False,False,False,False,False,False,False
3,29,33245,228,False,True,True,True,True,True,True,...,True,False,False,False,False,False,False,False,False,False
4,32,33245,306,False,True,True,True,True,True,True,...,True,False,False,False,False,False,False,False,False,False
5,34,43361,1238,True,True,False,False,True,True,False,...,False,False,True,False,False,False,False,False,False,False
6,39,13968,994,True,True,True,True,True,True,True,...,False,False,False,False,True,False,False,False,False,False
7,42,9055,67,False,True,True,True,True,True,True,...,True,False,False,False,False,False,False,False,True,False
8,43,33245,1432,True,True,True,True,True,True,True,...,False,False,False,False,False,False,True,False,False,False
9,74,18979,1759,True,True,True,True,True,True,True,...,True,False,True,False,False,False,False,False,False,False


In [169]:
elite.query("""SELECT f.name AS faction, stations.name, f.state, distance_to_star, type, max_landing_pad_size, is_planetary FROM stations
                INNER JOIN factions f on controlling_minor_faction_id = f.id
                LIMIT 20""")

,faction,name,state,distance_to_star,type,max_landing_pad_size,is_planetary
0,Abukunin Silver Fortune Ind,Reilly Hub,Civil War,171,Orbis Starport,L,False
1,Blood Brothers from Alrai,Bounds Hub,Boom,1080,Coriolis Starport,L,False
2,Civitas Dei,Starzl Plant,War,1834,Civilian Outpost,M,False
3,Beta Trianguli Australis Advanced Systems,Lunan Hub,Boom,228,Orbis Starport,L,False
4,Beta Trianguli Australis Advanced Systems,Hutchinson City,Boom,306,Orbis Starport,L,False
5,Future of Maijarua,Teng-hui Station,Boom,1238,Industrial Outpost,M,False
6,39 Tauri Interstellar,Porta,None,994,Orbis Starport,L,False
7,Agarla Citizens' Forum,Wilson Port,War,67,Coriolis Starport,L,False
8,Beta Trianguli Australis Advanced Systems,Vinge Gateway,Boom,1432,Orbis Starport,L,False
9,The Order of Mobius,Reamy Dock,Boom,1759,Coriolis Starport,L,False


In [170]:
systems.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20482 entries, 0 to 20481
Data columns (total 29 columns):
allegiance                      20472 non-null object
allegiance_id                   20472 non-null float64
controlling_minor_faction       20472 non-null object
controlling_minor_faction_id    20472 non-null float64
edsm_id                         20479 non-null float64
government                      20472 non-null object
government_id                   20472 non-null float64
id                              20482 non-null int64
is_populated                    20482 non-null bool
minor_faction_presences         20482 non-null object
name                            20482 non-null object
needs_permit                    20482 non-null bool
population                      20475 non-null float64
power                           8728 non-null object
power_state                     9140 non-null object
power_state_id                  9140 non-null float64
primary_economy              

In [172]:
has_market = mkt_stations['system_id'].unique()
system_drop = []
for i in systems.index:
    if systems.loc[i, 'id'] not in has_market:
        system_drop.append(i)
mkt_systems = systems.drop(system_drop)

In [173]:
mkt_systems.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19874 entries, 0 to 20480
Data columns (total 29 columns):
allegiance                      19871 non-null object
allegiance_id                   19871 non-null float64
controlling_minor_faction       19871 non-null object
controlling_minor_faction_id    19871 non-null float64
edsm_id                         19871 non-null float64
government                      19871 non-null object
government_id                   19871 non-null float64
id                              19874 non-null int64
is_populated                    19874 non-null bool
minor_faction_presences         19874 non-null object
name                            19874 non-null object
needs_permit                    19874 non-null bool
population                      19870 non-null float64
power                           8696 non-null object
power_state                     9107 non-null object
power_state_id                  9107 non-null float64
primary_economy              

In [181]:
mkt_systems[mkt_systems['controlling_minor_faction_id'].isnull()]['name']

1288       Asellus Primus
11285            LHS 3447
15997    Shinrarta Dezhra
Name: name, dtype: object

In [210]:
new_factions[new_factions['name'].str.contains('Pilot')]

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
618,5077,The Pilots Federation,1508070122,96,Democracy,4,Independent,80,None,16827,0
9537,74973,Brazilian Empire Pilots,1518967314,144,Patronage,2,Empire,64,Civil War,526,1
9608,75198,Indonesian Pilots Association,1519769267,96,Democracy,3,Federation,16,Boom,17252,0
9643,75276,8th Imperial Pilot's Syndicate,1519566906,150,Prison Colony,2,Empire,65,Election,18966,0
9703,75395,Asellus Primus Academy For Novitiate Pilots,1496077555,80,Cooperative,4,Independent,80,None,1365,1
9727,75428,LHS 3447 Academy for Novitiate Pilots,1507452303,80,Cooperative,4,Independent,80,None,12024,0
9760,75466,Free Spirit Pilots,1518462741,64,Corporate,4,Independent,72,Outbreak,13540,0
9851,75650,Pilots Ascension Program,1519680328,64,Corporate,4,Independent,80,None,1326,1
9890,75799,Pilots Federation Local Branch,1519392303,96,Democracy,7,Pilots Federation,80,None,16827,0
9913,75832,Sidhethe Uathne's Pilots' Trading Guild,1519363269,80,Cooperative,4,Independent,64,Civil War,8678,0


In [203]:
new_factions[new_factions['name'].isnull()]

,id,name,updated_at,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
49194,69320,NaN,1519760140,96,Democracy,4,Independent,80,None,12749,1


In [211]:
new_factions.loc[49194, 'name'] = 'Null'

In [212]:
new_factions.drop('updated_at', axis=1, inplace=True)

In [213]:
new_factions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75787 entries, 0 to 75786
Data columns (total 10 columns):
id                   75787 non-null int64
name                 75787 non-null object
government_id        75787 non-null int64
government           75787 non-null object
allegiance_id        75787 non-null int64
allegiance           75787 non-null object
state_id             75787 non-null int64
state                75787 non-null object
home_system_id       75787 non-null int64
is_player_faction    75787 non-null int64
dtypes: int64(6), object(4)
memory usage: 5.8+ MB


In [214]:
new_factions['is_player_faction'] = new_factions['is_player_faction'].astype(bool)

In [219]:
elite.insert("DROP TABLE factions;")

In [220]:
#new_factions.set_index('id',inplace=True)
#elite.upload(new_factions, "factions")

In [221]:
elite.query("""SELECT * FROM factions
               WHERE name = 'Null'""")

,id,name,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
0,69320,Null,96,Democracy,4,Independent,80,None,12749,True


In [222]:
mkt_systems.drop(['allegiance', 'allegiance_id', 'government', 'government_id', 'state', 'state_id'], axis=1, inplace=True)

In [225]:
mkt_systems.drop(['edsm_id', 'simbad_ref'], axis=1, inplace=True)

In [229]:
mkt_systems.drop(['power', 'power_state', 'power_state_id'], axis=1, inplace=True)

In [254]:
mkt_systems.drop(['reserve_type', 'reserve_type_id'], axis=1, inplace=True)

In [227]:
mkt_systems.drop(['updated_at'], axis=1, inplace=True)

In [237]:
mkt_systems.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19874 entries, 0 to 20480
Data columns (total 17 columns):
controlling_minor_faction       19871 non-null object
controlling_minor_faction_id    19874 non-null int64
id                              19874 non-null int64
is_populated                    19874 non-null bool
minor_faction_presences         19874 non-null object
name                            19874 non-null object
needs_permit                    19874 non-null bool
population                      19870 non-null float64
primary_economy                 19874 non-null object
primary_economy_id              19874 non-null float64
reserve_type                    9904 non-null object
reserve_type_id                 9904 non-null float64
security                        19874 non-null object
security_id                     19874 non-null float64
x                               19874 non-null float64
y                               19874 non-null float64
z                            

In [231]:
new_factions['allegiance'].unique()

array(['Independent', 'Federation', 'Empire', 'Alliance',
       'Pilots Federation'], dtype=object)

In [232]:
new_factions[new_factions['allegiance'] == 'Pilots Federation']

,name,government_id,government,allegiance_id,allegiance,state_id,state,home_system_id,is_player_faction
id,,,,,,,,,
75799,Pilots Federation Local Branch,96,Democracy,7,Pilots Federation,80,None,16827,False


In [235]:
mkt_systems['controlling_minor_faction_id'].fillna(75799, inplace=True)

In [236]:
mkt_systems['controlling_minor_faction_id'] = mkt_systems['controlling_minor_faction_id'].astype(int)

In [238]:
mkt_systems['controlling_minor_faction'].fillna(new_factions.loc[75799, 'name'], inplace=True)

In [239]:
mkt_systems[mkt_systems['controlling_minor_faction_id'] == 75799]

,controlling_minor_faction,controlling_minor_faction_id,id,is_populated,minor_faction_presences,name,needs_permit,population,primary_economy,primary_economy_id,reserve_type,reserve_type_id,security,security_id,x,y,z
1288,Pilots Federation Local Branch,75799,1365,True,"[{'minor_faction_id': 1481, 'state_id': 69, 'i...",Asellus Primus,False,5.000000e+04,Extraction,2.0,Common,3.0,Medium,32.0,-23.93750,40.87500,-1.34375
11285,Pilots Federation Local Branch,75799,12024,True,"[{'minor_faction_id': 75428, 'state_id': 80, '...",LHS 3447,False,1.148206e+09,Agriculture,1.0,Common,3.0,High,48.0,-43.18750,-5.28125,56.15625
15997,Pilots Federation Local Branch,75799,16827,True,"[{'minor_faction_id': 75300, 'state_id': 80, '...",Shinrarta Dezhra,True,8.520694e+07,High Tech,3.0,Common,3.0,High,48.0,55.71875,17.59375,27.15625


In [240]:
mkt_systems[mkt_systems['population'].isnull()]

,controlling_minor_faction,controlling_minor_faction_id,id,is_populated,minor_faction_presences,name,needs_permit,population,primary_economy,primary_economy_id,reserve_type,reserve_type_id,security,security_id,x,y,z
19718,Gold Dynamic PLC,38765,217315,True,"[{'minor_faction_id': 38765, 'state_id': 80, '...",HIP 85038,False,NaN,Extraction,2.0,None,NaN,Low,16.0,90.15625,-45.06250,176.31250
20326,Dedalo Corp. Interstellar,25650,4189983,True,"[{'minor_faction_id': 25650, 'state_id': 16, '...",Hydrae Sector ST-R b4-1,False,NaN,Refinery,6.0,Pristine,1.0,Medium,32.0,97.59375,83.40625,113.15625
20343,Mu Kin Co-op,1635,4461319,True,"[{'minor_faction_id': 1635, 'state_id': 16, 'i...",Pegasi Sector BQ-Y d94,False,NaN,Colony,11.0,None,NaN,Low,16.0,-172.50000,-73.18750,-53.59375
20345,Jet Life Industry,21119,4491961,True,"[{'minor_faction_id': 21119, 'state_id': 80, '...",Col 285 Sector EN-I c10-12,False,NaN,Extraction,2.0,None,NaN,Low,16.0,-116.21875,115.34375,86.03125


In [249]:
mkt_systems.loc[19718, 'population'] = 572
mkt_systems.loc[20326, 'population'] = 382514
mkt_systems.loc[20343, 'population'] = 545
mkt_systems.loc[20345, 'population'] = 3450

In [250]:
mkt_systems['population'] = mkt_systems['population'].astype(int)

In [255]:
mkt_systems.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19874 entries, 0 to 20480
Data columns (total 15 columns):
controlling_minor_faction       19874 non-null object
controlling_minor_faction_id    19874 non-null int64
id                              19874 non-null int64
is_populated                    19874 non-null bool
minor_faction_presences         19874 non-null object
name                            19874 non-null object
needs_permit                    19874 non-null bool
population                      19874 non-null int64
primary_economy                 19874 non-null object
primary_economy_id              19874 non-null float64
security                        19874 non-null object
security_id                     19874 non-null float64
x                               19874 non-null float64
y                               19874 non-null float64
z                               19874 non-null float64
dtypes: bool(2), float64(5), int64(3), object(5)
memory usage: 2.8+ MB


In [257]:
mkt_systems['primary_economy_id'] = mkt_systems['primary_economy_id'].astype(int)
mkt_systems['security_id'] = mkt_systems['security_id'].astype(int)

In [278]:
faction_inf = []
for fct in mkt_systems.loc[0, 'minor_faction_presences']:
    if fct['influence'] != None:
        faction_inf.append(fct['influence'])
faction_inf.sort(reverse=True)
print(faction_inf[0])
for fct in mkt_systems.loc[0, 'minor_faction_presences']:
    if fct['influence'] == faction_inf[1]:
        print(fct['influence'])
        print(fct['minor_faction_id'])

42.4182
14.4698
40897


In [287]:
mkt_systems['controlling_minor_faction_inf'] = 99.0
mkt_systems['secondary_minor_faction_id'] = 0
mkt_systems['secondary_minor_faction_inf'] = 1.0
missing_2nd_fac = []
for i in mkt_systems.index:
    faction_inf = []
    for fct in mkt_systems.loc[i, 'minor_faction_presences']:
        if fct['influence'] != None:
            faction_inf.append(fct['influence'])
    faction_inf.sort(reverse=True)
    mkt_systems.loc[i, 'controlling_minor_faction_inf'] = faction_inf[0]
    for fct in mkt_systems.loc[i, 'minor_faction_presences']:
        if len(faction_inf) > 1:
            if fct['influence'] == faction_inf[1]:
                mkt_systems.loc[i, 'secondary_minor_faction_inf'] = fct['influence']
                mkt_systems.loc[i, 'secondary_minor_faction_id'] = fct['minor_faction_id']
        else:
            missing_2nd_fac.append(i)
            mkt_systems.loc[i, 'secondary_minor_faction_inf'] = 0
            mkt_systems.loc[i, 'secondary_minor_faction_id'] = 0

In [290]:
for i in missing_2nd_fac:
    print(mkt_systems.loc[i, 'minor_faction_presences'])

[{'minor_faction_id': 75335, 'state_id': 16, 'influence': 100, 'state': 'Boom'}]
[{'minor_faction_id': 75366, 'state_id': 16, 'influence': 100, 'state': 'Boom'}]
[{'minor_faction_id': 75148, 'state_id': 16, 'influence': 100, 'state': 'Boom'}]
[{'minor_faction_id': 75148, 'state_id': 16, 'influence': 100, 'state': 'Boom'}]
[{'minor_faction_id': 75148, 'state_id': 16, 'influence': 100, 'state': 'Boom'}]
[{'minor_faction_id': 74880, 'state_id': 96, 'influence': 100, 'state': 'Retreat'}]
[{'minor_faction_id': 75371, 'state_id': 16, 'influence': 100, 'state': 'Boom'}]
[{'minor_faction_id': 75148, 'state_id': 16, 'influence': 100, 'state': 'Boom'}]
[{'minor_faction_id': 75105, 'state_id': 80, 'influence': 100, 'state': 'None'}]


In [266]:
x = mkt_systems[mkt_systems['name'] == 'Epsilon Indi']['x']
y = mkt_systems[mkt_systems['name'] == 'Epsilon Indi']['y']
z = mkt_systems[mkt_systems['name'] == 'Epsilon Indi']['z']

np.sqrt(x**2 + y**2 + z**2)
#distance function works as expected

4284    11.80241
dtype: float64

In [291]:
mkt_systems.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19874 entries, 0 to 20480
Data columns (total 18 columns):
controlling_minor_faction        19874 non-null object
controlling_minor_faction_id     19874 non-null int64
id                               19874 non-null int64
is_populated                     19874 non-null bool
minor_faction_presences          19874 non-null object
name                             19874 non-null object
needs_permit                     19874 non-null bool
population                       19874 non-null int64
primary_economy                  19874 non-null object
primary_economy_id               19874 non-null int64
security                         19874 non-null object
security_id                      19874 non-null int64
x                                19874 non-null float64
y                                19874 non-null float64
z                                19874 non-null float64
controlling_minor_faction_inf    19874 non-null float64
secondary_minor_

In [293]:
for i in mkt_systems.index:
    if mkt_systems.loc[i, 'controlling_minor_faction_inf'] <= mkt_systems.loc[i, 'secondary_minor_faction_inf']:
        print(i)
        print(mkt_systems.loc[i, 'controlling_minor_faction_inf'])
        print(mkt_systems.loc[i, 'secondary_minor_faction_inf'])

5
41.1588
41.1588
51
22.6226
22.6226
133
21.4
21.4
258
23.3233
23.3233
345
26.5469
26.5469
431
44.1
44.1
432
31.4
31.4
443
23.5
23.5
457
39.5
39.5
506
38.7387
38.7387
568
26.0
26.0
622
26.7267
26.7267
809
28.8
28.8
1011
22.9229
22.9229
1116
32.2
32.2
1291
33.1
33.1
1346
26.6
26.6
1408
20.7207
20.7207
1429
36.9
36.9
1477
31.4
31.4
1505
30.4262
30.4262
1543
35.4
35.4
1577
23.6236
23.6236
1586
32.4324
32.4324
1747
36.5
36.5
1817
21.5215
21.5215
2093
21.6216
21.6216
2402
42.8
42.8
2618
39.0
39.0
2686
23.7762
23.7762
3139
29.1708
29.1708
3196
18.6
18.6
3231
29.8
29.8
3426
19.9
19.9
3462
29.97
29.97
3463
47.1
47.1
3684
32.1
32.1
3719
21.6783
21.6783
3762
27.9279
27.9279
3838
42.9
42.9
3857
35.6
35.6
3944
36.4
36.4
3986
44.5445
44.5445
4125
29.8298
29.8298
4310
33.966
33.966
4424
35.8
35.8
4450
32.3
32.3
4512
41.4414
41.4414
4538
35.0649
35.0649
4653
27.0
27.0
4663
36.5634
36.5634
4695
25.2747
25.2747
4715
38.3
38.3
4819
30.4304
30.4304
5008
32.8671
32.8671
5180
26.2
26.2
5268
23.8238
23.8238

In [295]:
mkt_systems.loc[5]['minor_faction_presences']
#nope, that's just how it is

[{'influence': 41.1588,
  'minor_faction_id': 7167,
  'state': 'Boom',
  'state_id': 16},
 {'influence': 0.999,
  'minor_faction_id': 7169,
  'state': 'None',
  'state_id': 80},
 {'influence': 41.1588,
  'minor_faction_id': 7170,
  'state': 'Boom',
  'state_id': 16},
 {'influence': 6.6933,
  'minor_faction_id': 27184,
  'state': 'None',
  'state_id': 80},
 {'influence': 9.99,
  'minor_faction_id': 27179,
  'state': 'None',
  'state_id': 80}]

In [297]:
minor_faction_presences = mkt_systems['minor_faction_presences']
mkt_systems.drop('minor_faction_presences', axis=1, inplace=True)

In [298]:
mkt_systems.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19874 entries, 0 to 20480
Data columns (total 17 columns):
controlling_minor_faction        19874 non-null object
controlling_minor_faction_id     19874 non-null int64
id                               19874 non-null int64
is_populated                     19874 non-null bool
name                             19874 non-null object
needs_permit                     19874 non-null bool
population                       19874 non-null int64
primary_economy                  19874 non-null object
primary_economy_id               19874 non-null int64
security                         19874 non-null object
security_id                      19874 non-null int64
x                                19874 non-null float64
y                                19874 non-null float64
z                                19874 non-null float64
controlling_minor_faction_inf    19874 non-null float64
secondary_minor_faction_id       19874 non-null int64
secondary_minor_f

In [299]:
mkt_systems.head()

,controlling_minor_faction,controlling_minor_faction_id,id,is_populated,name,needs_permit,population,primary_economy,primary_economy_id,security,security_id,x,y,z,controlling_minor_faction_inf,secondary_minor_faction_id,secondary_minor_faction_inf
0,1 G. Caeli Empire League,31816,1,True,1 G. Caeli,False,6544826,Industrial,4,Medium,32,80.90625,-83.53125,-30.81250,42.4182,40897,14.4698
1,1 Geminorum Liberals,56677,2,True,1 Geminorum,False,141727,Terraforming,8,Medium,32,19.78125,3.56250,-153.81250,54.9000,26454,14.8000
2,Values Party of 1 Hydrae,26307,3,True,1 Hydrae,False,6028981745,Agriculture,1,High,48,60.90625,28.53125,-54.90625,36.4000,42316,33.3000
3,1 i Centauri Values Party,7694,4,True,1 i Centauri,False,3462462067,Agriculture,1,Medium,32,38.46875,29.78125,40.46875,37.9379,7696,20.2202
4,1 Kappa Cygni Movement,68556,5,True,1 Kappa Cygni,False,24843190,Industrial,4,Medium,32,-117.75000,37.78125,11.18750,74.4000,68556,17.5000


In [300]:
mkt_systems['is_populated'].unique()

array([ True])

In [301]:
mkt_systems.drop('is_populated', axis=1, inplace=True)

In [302]:
mkt_systems.loc[0]

controlling_minor_faction        1 G. Caeli Empire League
controlling_minor_faction_id                        31816
id                                                      1
name                                           1 G. Caeli
needs_permit                                        False
population                                        6544826
primary_economy                                Industrial
primary_economy_id                                      4
security                                           Medium
security_id                                            32
x                                                 80.9062
y                                                -83.5312
z                                                -30.8125
controlling_minor_faction_inf                     42.4182
secondary_minor_faction_id                          40897
secondary_minor_faction_inf                       14.4698
Name: 0, dtype: object

In [304]:
mkt_systems.set_index('id', inplace=True)

In [305]:
#elite.upload(mkt_systems, "systems")

In [306]:
elite.query("""SELECT * FROM systems
               LIMIT 20""")

,id,controlling_minor_faction,controlling_minor_faction_id,name,needs_permit,population,primary_economy,primary_economy_id,security,security_id,x,y,z,controlling_minor_faction_inf,secondary_minor_faction_id,secondary_minor_faction_inf
0,1,1 G. Caeli Empire League,31816,1 G. Caeli,False,6544826,Industrial,4,Medium,32,80.90625,-83.53125,-30.81250,42.4182,40897,14.4698
1,2,1 Geminorum Liberals,56677,1 Geminorum,False,141727,Terraforming,8,Medium,32,19.78125,3.56250,-153.81250,54.9000,26454,14.8000
2,3,Values Party of 1 Hydrae,26307,1 Hydrae,False,6028981745,Agriculture,1,High,48,60.90625,28.53125,-54.90625,36.4000,42316,33.3000
3,4,1 i Centauri Values Party,7694,1 i Centauri,False,3462462067,Agriculture,1,Medium,32,38.46875,29.78125,40.46875,37.9379,7696,20.2202
4,5,1 Kappa Cygni Movement,68556,1 Kappa Cygni,False,24843190,Industrial,4,Medium,32,-117.75000,37.78125,11.18750,74.4000,68556,17.5000
5,6,Friends of 10 Arietis,7170,10 Arietis,False,570041,Refinery,6,Low,16,-79.09375,-88.21875,-105.87500,41.1588,7170,41.1588
6,7,10 Canum Venaticorum Bridge Ltd,49631,10 Canum Venaticorum,False,1549685657,Agriculture,1,High,48,-9.37500,55.43750,-7.00000,36.8000,38421,24.7000
7,9,Values Party of 10 G. Canis Majoris,23375,10 G. Canis Majoris,False,63317571,Terraforming,8,Medium,32,96.56250,-38.28125,-80.31250,65.3654,23377,12.1121
8,10,10 Hydrae Noblement,511,10 Hydrae,False,34803,Extraction,2,Medium,32,123.46875,99.78125,-138.31250,60.4000,17955,10.3000
9,11,10 Iota Cygni Front,44419,10 Iota Cygni,False,14279056,Industrial,4,High,48,-116.37500,32.03125,12.62500,54.9451,44416,14.6853


In [308]:
elite.query("""SELECT s.name as station, f.state, sys.name as system, c.name as commodity, c.category, supply, buy_price, sell_price, demand FROM listings
               INNER JOIN commodities c ON commodity_id = c.id
               INNER JOIN stations s ON station_id = s.id
               INNER JOIN systems sys ON s.system_id = sys.id
               INNER JOIN factions f ON s.controlling_minor_faction_id = f.id
               WHERE sys.name = 'Sol'
               LIMIT 20""")

,station,state,system,commodity,category,supply,buy_price,sell_price,demand
0,M.Gorbachev,Boom,Sol,Hydrogen Fuel,Chemicals,1696094,96,91,0
1,M.Gorbachev,Boom,Sol,Clothing,Consumer Items,0,0,768,994590
2,M.Gorbachev,Boom,Sol,Consumer Technology,Consumer Items,0,0,6849,71979
3,M.Gorbachev,Boom,Sol,Domestic Appliances,Consumer Items,0,0,1003,322647
4,M.Gorbachev,Boom,Sol,Beer,Legal Drugs,0,0,609,880760
5,M.Gorbachev,Boom,Sol,Liquor,Legal Drugs,0,0,1109,58769
6,M.Gorbachev,Boom,Sol,Wine,Legal Drugs,0,0,692,759151
7,M.Gorbachev,Boom,Sol,Power Generators,Machinery,0,0,527,5588
8,M.Gorbachev,Boom,Sol,Water Purifiers,Machinery,0,0,457,146470
9,M.Gorbachev,Boom,Sol,Basic Medicines,Medicines,0,0,453,40350
